In [1]:
# %reset

### https://github.com/plotly/dash-sample-apps

In [2]:
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph
# https://stackoverflow.com/questions/63714679/plotting-gannt-chart-using-timestamps
# https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587

# ******** open this
# https://community.plotly.com/t/how-do-i-change-the-x-axis-representation-of-time-on-the-px-timeline/44834/4

# https://plotly.com/python/gantt/
# https://plotly.com/python-api-reference/generated/plotly.express.timeline.html
# https://plotly.com/python/reference/layout/yaxis/

# for when you want to reduce the number of dimensions of a plot, 
# i.e. make another chart with the dimension separated out, i.e. multiple charts of different y-variables
# https://plotly.com/python/facet-plots/
# *******

import numpy as np
import pandas as pd
import plotly.express as px

import jupyter_dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import datetime as dt, date, time, timedelta

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# report = '~/qs/time/data/report-01-01-2021.csv'
report = '~/qs/time/data/report-01-01-2022.csv'

df = pd.read_csv(report)

In [3]:
original_df = df = df[['Group', 'Activity type', 'From', 'To']]
original_df

,Group,Activity type,From,To
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 5:43:59
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54
...,...,...,...,...
9431,NaN,Toilet,2021-01-01 8:10:38,2021-01-01 8:12:25
9432,Computer,Games,2021-01-01 0:47:12,2021-01-01 8:10:35
9433,Computer,Internet,2021-01-01 0:42:17,2021-01-01 0:47:11
9434,NaN,Toilet,2021-01-01 0:37:35,2021-01-01 0:38:40


In [4]:
original_df = df = df.rename(columns={'Activity type':'Activity', 
                        'From':'Start', 
                        'To':'End'})
original_df

,Group,Activity,Start,End
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 5:43:59
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54
...,...,...,...,...
9431,NaN,Toilet,2021-01-01 8:10:38,2021-01-01 8:12:25
9432,Computer,Games,2021-01-01 0:47:12,2021-01-01 8:10:35
9433,Computer,Internet,2021-01-01 0:42:17,2021-01-01 0:47:11
9434,NaN,Toilet,2021-01-01 0:37:35,2021-01-01 0:38:40


In [5]:
from IPython.display import display
with pd.option_context('display.max_rows', 50, 'display.max_columns', 5):
    display(df[-10:])

,Group,Activity,Start,End
9426,NaN,Grooming,2021-01-01 12:35:01,2021-01-01 12:37:53
9427,NaN,Toilet,2021-01-01 12:28:07,2021-01-01 12:32:09
9428,Intimate,Masturbation,2021-01-01 12:17:14,2021-01-01 12:27:55
9429,NaN,Eating,2021-01-01 8:13:08,2021-01-01 8:32:50
9430,Computer,Internet,2021-01-01 8:13:03,2021-01-01 12:17:12
9431,NaN,Toilet,2021-01-01 8:10:38,2021-01-01 8:12:25
9432,Computer,Games,2021-01-01 0:47:12,2021-01-01 8:10:35
9433,Computer,Internet,2021-01-01 0:42:17,2021-01-01 0:47:11
9434,NaN,Toilet,2021-01-01 0:37:35,2021-01-01 0:38:40
9435,Computer,Internet,2020-12-31 23:30:41,2021-01-01 0:37:23


In [6]:
# # for 2021
# #df = df.drop(df.index[-37:])

# # for 2020
# # df = df.drop(df.index[8728:])
# original_df = df = df.drop(df.index[-36:])

# original_df.tail(5)

In [7]:
start = original_df['Start']
# end = original_df['End']
# activity = original_df['Activity']
df['Day'] = start.str[0:10]
# df['Day'] = df['Start'].dt.date

# day = original_df['Day']

original_df

,Group,Activity,Start,End,Day
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 5:43:59,2021-12-31
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31
...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 8:10:38,2021-01-01 8:12:25,2021-01-01
9432,Computer,Games,2021-01-01 0:47:12,2021-01-01 8:10:35,2021-01-01
9433,Computer,Internet,2021-01-01 0:42:17,2021-01-01 0:47:11,2021-01-01
9434,NaN,Toilet,2021-01-01 0:37:35,2021-01-01 0:38:40,2021-01-01


In [8]:
original_df['Start'] = df['Start'] = pd.to_datetime(df['Start'])
original_df['End'] = df['End'] = pd.to_datetime(df['End'])

original_df

,Group,Activity,Start,End,Day
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 05:43:59,2021-12-31
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31
...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 08:10:38,2021-01-01 08:12:25,2021-01-01
9432,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01
9433,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01
9434,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01


# Replacing activity words for other words

In [9]:
# for activity in df['Activity'].unique():
#     print(activity)

In [10]:
# def replace_activity_name(df=df):
#     for activity in df['Activity'].unique():
#         print(activity)
#     print()

#     while True:
#         replace = input("Do you want to replace any activity's names with something else? (y/n): \n")
#         try:
#             if replace == 'y' or replace == 'Y' or replace == 'yes' or replace == 'Yes' :
#                 replacement_activity = input('What activity do you want to replace?: ')
#                 replaced_with = input('And what do you want to replace it with?: ')
#                 df.loc[df['Activity'].str.contains(replacement_activity), 'Activity'] = replaced_with
#                 original_df.loc[df['Activity'].str.contains(replacement_activity), 'Activity'] = replaced_with
#                 print(f'Replaced: {replacement_activity} with {replaced_with}')
#                 continue
#             elif replace == "n" or replace == "N" or replace == "no" or replace == "No" or replace == "" or replace == " ":
#                 print("Finished replacing activity names.")
#             else:
#                 raise Exception("Answer must be some variation of 'yes' or 'no.'")

#         except Exception as e:
#             print(e)

#         else:
#             for activity in df['Activity'].unique():
#                 print(activity)
#             break

In [11]:
# replace_activity_name(df)

In [12]:
replace_activity = 'Masturbation'
df.loc[df['Activity'].str.contains(replace_activity), 'Activity'] = 'Personal Time'
original_df.loc[df['Activity'].str.contains(replace_activity), 'Activity'] = 'Personal Time'

original_df

,Group,Activity,Start,End,Day
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 05:43:59,2021-12-31
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31
...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 08:10:38,2021-01-01 08:12:25,2021-01-01
9432,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01
9433,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01
9434,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01


In [13]:
replace_activity = 'Sex'
df.loc[df['Activity'].str.contains(replace_activity), 'Activity'] = 'Personal Time'
original_df.loc[df['Activity'].str.contains(replace_activity), 'Activity'] = 'Personal Time'

original_df

,Group,Activity,Start,End,Day
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 05:43:59,2021-12-31
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31
...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 08:10:38,2021-01-01 08:12:25,2021-01-01
9432,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01
9433,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01
9434,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01


# To get duration of original activities (in \<seconds>)

In [14]:
original_df['Duration in Seconds'] = df['Duration in Seconds'] = pd.to_timedelta(df['End'] - df['Start']).astype('timedelta64[s]')
original_df

,Group,Activity,Start,End,Day,Duration in Seconds
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 05:43:59,2021-12-31,25821.0
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31,141.0
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31,62.0
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31,89.0
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31,14243.0
...,...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 08:10:38,2021-01-01 08:12:25,2021-01-01,107.0
9432,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01,26603.0
9433,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01,294.0
9434,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01,65.0


In [15]:
# https://stackoverflow.com/questions/55962886/convert-column-of-seconds-to-hhmmss-pandas/55962938
original_df['Duration'] = pd.to_datetime(df["Duration in Seconds"], unit='s').dt.strftime("%H:%M:%S")
original_df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 05:43:59,2021-12-31,25821.0,07:10:21
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31,141.0,00:02:21
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31,62.0,00:01:02
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31,89.0,00:01:29
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31,14243.0,03:57:23
...,...,...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 08:10:38,2021-01-01 08:12:25,2021-01-01,107.0,00:01:47
9432,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01,26603.0,07:23:23
9433,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01,294.0,00:04:54
9434,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01,65.0,00:01:05


# Start of splitting activities at midnight

In [16]:
# https://stackoverflow.com/questions/59791843/pandas-splitting-date-ranges-on-specific-day-boundary
# https://stackoverflow.com/questions/50336251/split-dataframe-entries-at-midnight
edges = pd.date_range(df.Start.min().normalize() - pd.Timedelta(days=1),
                      df.End.max().normalize() + pd.Timedelta(days=1), freq='D')
edges

DatetimeIndex(['2020-12-30', '2020-12-31', '2021-01-01', '2021-01-02',
               '2021-01-03', '2021-01-04', '2021-01-05', '2021-01-06',
               '2021-01-07', '2021-01-08',
               ...
               '2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27',
               '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31',
               '2022-01-01', '2022-01-02'],
              dtype='datetime64[ns]', length=369, freq='D')

In [17]:
l = []
for edge in edges:
    m = df.Start.lt(edge) & df.End.gt(edge)            # Rows to modify
    l.append(df.loc[m].assign(End=edge))               # Clip end of modified rows
    df.loc[m, 'Start'] = edge                          # Fix start for next edge

    
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
df = pd.concat(l+[df], ignore_index=True).sort_values('Start')
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,Computer,Internet,2020-12-31 23:30:41,2021-01-01 00:00:00,2020-12-31,4002.0,01:06:42
9853,Computer,Internet,2021-01-01 00:00:00,2021-01-01 00:37:23,2020-12-31,4002.0,01:06:42
9852,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01,65.0,00:01:05
9851,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01,294.0,00:04:54
9850,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01,26603.0,07:23:23
...,...,...,...,...,...,...,...
421,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31,89.0,00:01:29
420,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31,62.0,00:01:02
419,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31,141.0,00:02:21
417,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 00:00:00,2021-12-31,25821.0,07:10:21


In [18]:
df['Day'] = df['Start'].dt.date
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,Computer,Internet,2020-12-31 23:30:41,2021-01-01 00:00:00,2020-12-31,4002.0,01:06:42
9853,Computer,Internet,2021-01-01 00:00:00,2021-01-01 00:37:23,2021-01-01,4002.0,01:06:42
9852,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01,65.0,00:01:05
9851,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01,294.0,00:04:54
9850,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01,26603.0,07:23:23
...,...,...,...,...,...,...,...
421,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31,89.0,00:01:29
420,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31,62.0,00:01:02
419,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31,141.0,00:02:21
417,NaN,Sleeping,2021-12-31 22:33:38,2022-01-01 00:00:00,2021-12-31,25821.0,07:10:21


In [19]:
# https://stackoverflow.com/questions/28888730/pandas-change-day
# https://stackoverflow.com/questions/26882499/reset-time-part-of-a-pandas-timestamp
# https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.replace.html

# https://stackoverflow.com/questions/100210/what-is-the-standard-way-to-add-n-seconds-to-datetime-time-in-python/100345

df['Start'] = pd.to_datetime(df.Start)
df["Start"] = df["Start"].apply(lambda x: x.replace(year=1970, month=1, day=1))
df['Start'] = pd.to_datetime(df.Start)

df['End'] = pd.to_datetime(df.End)
df["End"] = df["End"].apply(lambda x: x.replace(year=1970, month=1, day=1))
df['End'] = pd.to_datetime(df.End)

df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,Computer,Internet,1970-01-01 23:30:41,1970-01-01 00:00:00,2020-12-31,4002.0,01:06:42
9853,Computer,Internet,1970-01-01 00:00:00,1970-01-01 00:37:23,2021-01-01,4002.0,01:06:42
9852,NaN,Toilet,1970-01-01 00:37:35,1970-01-01 00:38:40,2021-01-01,65.0,00:01:05
9851,Computer,Internet,1970-01-01 00:42:17,1970-01-01 00:47:11,2021-01-01,294.0,00:04:54
9850,Computer,Games,1970-01-01 00:47:12,1970-01-01 08:10:35,2021-01-01,26603.0,07:23:23
...,...,...,...,...,...,...,...
421,NaN,Eating,1970-01-01 18:32:06,1970-01-01 18:33:35,2021-12-31,89.0,00:01:29
420,NaN,Toilet,1970-01-01 22:28:58,1970-01-01 22:30:00,2021-12-31,62.0,00:01:02
419,NaN,Grooming,1970-01-01 22:30:00,1970-01-01 22:32:21,2021-12-31,141.0,00:02:21
417,NaN,Sleeping,1970-01-01 22:33:38,1970-01-01 00:00:00,2021-12-31,25821.0,07:10:21


In [20]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mask.html

m = dt(year=1970, month=1, day=1, hour=0, minute=0, second=0)
new = dt(year=1970, month=1, day=1, hour=0, minute=0, second=1)

df['Start'] = df['Start'].mask(df['Start'] == m, new)

In [21]:
m = dt(year=1970, month=1, day=1, hour=0, minute=0, second=0)
new = dt(year=1970, month=1, day=1, hour=23, minute=59, second=59)

df['End'] = df['End'].mask(df['End'] == m, new)
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,Computer,Internet,1970-01-01 23:30:41,1970-01-01 23:59:59,2020-12-31,4002.0,01:06:42
9853,Computer,Internet,1970-01-01 00:00:01,1970-01-01 00:37:23,2021-01-01,4002.0,01:06:42
9852,NaN,Toilet,1970-01-01 00:37:35,1970-01-01 00:38:40,2021-01-01,65.0,00:01:05
9851,Computer,Internet,1970-01-01 00:42:17,1970-01-01 00:47:11,2021-01-01,294.0,00:04:54
9850,Computer,Games,1970-01-01 00:47:12,1970-01-01 08:10:35,2021-01-01,26603.0,07:23:23
...,...,...,...,...,...,...,...
421,NaN,Eating,1970-01-01 18:32:06,1970-01-01 18:33:35,2021-12-31,89.0,00:01:29
420,NaN,Toilet,1970-01-01 22:28:58,1970-01-01 22:30:00,2021-12-31,62.0,00:01:02
419,NaN,Grooming,1970-01-01 22:30:00,1970-01-01 22:32:21,2021-12-31,141.0,00:02:21
417,NaN,Sleeping,1970-01-01 22:33:38,1970-01-01 23:59:59,2021-12-31,25821.0,07:10:21


# To get duration of split times (in \<seconds>)

In [22]:
# https://stackoverflow.com/questions/49853295/calculate-cumulative-duration-of-a-pandas-datetime-column
df['Duration in Seconds'] = pd.to_timedelta(df['End'] - df['Start']).astype('timedelta64[s]')
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,Computer,Internet,1970-01-01 23:30:41,1970-01-01 23:59:59,2020-12-31,1758.0,01:06:42
9853,Computer,Internet,1970-01-01 00:00:01,1970-01-01 00:37:23,2021-01-01,2242.0,01:06:42
9852,NaN,Toilet,1970-01-01 00:37:35,1970-01-01 00:38:40,2021-01-01,65.0,00:01:05
9851,Computer,Internet,1970-01-01 00:42:17,1970-01-01 00:47:11,2021-01-01,294.0,00:04:54
9850,Computer,Games,1970-01-01 00:47:12,1970-01-01 08:10:35,2021-01-01,26603.0,07:23:23
...,...,...,...,...,...,...,...
421,NaN,Eating,1970-01-01 18:32:06,1970-01-01 18:33:35,2021-12-31,89.0,00:01:29
420,NaN,Toilet,1970-01-01 22:28:58,1970-01-01 22:30:00,2021-12-31,62.0,00:01:02
419,NaN,Grooming,1970-01-01 22:30:00,1970-01-01 22:32:21,2021-12-31,141.0,00:02:21
417,NaN,Sleeping,1970-01-01 22:33:38,1970-01-01 23:59:59,2021-12-31,5181.0,07:10:21


In [23]:
# https://stackoverflow.com/questions/55962886/convert-column-of-seconds-to-hhmmss-pandas/55962938
df['Duration'] = pd.to_datetime(df["Duration in Seconds"], unit='s').dt.strftime("%H:%M:%S")
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,Computer,Internet,1970-01-01 23:30:41,1970-01-01 23:59:59,2020-12-31,1758.0,00:29:18
9853,Computer,Internet,1970-01-01 00:00:01,1970-01-01 00:37:23,2021-01-01,2242.0,00:37:22
9852,NaN,Toilet,1970-01-01 00:37:35,1970-01-01 00:38:40,2021-01-01,65.0,00:01:05
9851,Computer,Internet,1970-01-01 00:42:17,1970-01-01 00:47:11,2021-01-01,294.0,00:04:54
9850,Computer,Games,1970-01-01 00:47:12,1970-01-01 08:10:35,2021-01-01,26603.0,07:23:23
...,...,...,...,...,...,...,...
421,NaN,Eating,1970-01-01 18:32:06,1970-01-01 18:33:35,2021-12-31,89.0,00:01:29
420,NaN,Toilet,1970-01-01 22:28:58,1970-01-01 22:30:00,2021-12-31,62.0,00:01:02
419,NaN,Grooming,1970-01-01 22:30:00,1970-01-01 22:32:21,2021-12-31,141.0,00:02:21
417,NaN,Sleeping,1970-01-01 22:33:38,1970-01-01 23:59:59,2021-12-31,5181.0,01:26:21


In [24]:
original_df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,NaN,Sleeping,2022-01-01 00:00:00,2022-01-01 05:43:59,2021-12-31,25821.0,07:10:21
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31,141.0,00:02:21
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31,62.0,00:01:02
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31,89.0,00:01:29
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31,14243.0,03:57:23
...,...,...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 08:10:38,2021-01-01 08:12:25,2021-01-01,107.0,00:01:47
9432,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01,26603.0,07:23:23
9433,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01,294.0,00:04:54
9434,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01,65.0,00:01:05


In [25]:
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,Computer,Internet,1970-01-01 23:30:41,1970-01-01 23:59:59,2020-12-31,1758.0,00:29:18
9853,Computer,Internet,1970-01-01 00:00:01,1970-01-01 00:37:23,2021-01-01,2242.0,00:37:22
9852,NaN,Toilet,1970-01-01 00:37:35,1970-01-01 00:38:40,2021-01-01,65.0,00:01:05
9851,Computer,Internet,1970-01-01 00:42:17,1970-01-01 00:47:11,2021-01-01,294.0,00:04:54
9850,Computer,Games,1970-01-01 00:47:12,1970-01-01 08:10:35,2021-01-01,26603.0,07:23:23
...,...,...,...,...,...,...,...
421,NaN,Eating,1970-01-01 18:32:06,1970-01-01 18:33:35,2021-12-31,89.0,00:01:29
420,NaN,Toilet,1970-01-01 22:28:58,1970-01-01 22:30:00,2021-12-31,62.0,00:01:02
419,NaN,Grooming,1970-01-01 22:30:00,1970-01-01 22:32:21,2021-12-31,141.0,00:02:21
417,NaN,Sleeping,1970-01-01 22:33:38,1970-01-01 23:59:59,2021-12-31,5181.0,01:26:21


# To export dataset

In [26]:
# df.to_csv('data/split_activities.csv', index=None, encoding='utf-8')

# Unique Activities

In [27]:
unique_activities = df['Activity'].unique()
unique_activities = unique_activities.tolist()
#unique_activities = sorted(unique_activities)

print(f"Num unique activities: {len(unique_activities)}")
print()
for i in unique_activities:
    print(i)

Num unique activities: 33

Internet
Toilet
Games
Eating
Personal Time
Grooming
Sleeping
Walking
Working
Programming
Cooking
Reading
Driving
Errands and Appointments
Family Time
Shopping
Cleaning
Writing
Dishes
Friends Time
Drinking
Drawing
Drumming
Dating
Movies/TV
Gym
Swimming/Tanning
Phone
Concert/Event
Dancing
Studying
Class
Biking


# Start of making colors

In [28]:
# import plotly.io as pio
# pio.renderers

In [29]:
# # https://plotly.com/python/builtin-colorscales/
# import plotly.express as px
# from textwrap import wrap

# named_colorscales = px.colors.named_colorscales()
# print('Named Colorscales:')
# print("\n".join(wrap("".join('{:<12}'.format(c) for c in named_colorscales), 96)))

# print()
# print(f'Plotly Colorscale: {px.colors.qualitative.Plotly}')

In [30]:
# # https://stackoverflow.com/questions/63460213/plotly-how-to-define-colors-in-a-figure-using-plotly-graph-objects-and-plotly-e
# # https://stackoverflow.com/questions/63604342/plotly-how-to-set-up-a-color-palette-for-a-figure-created-with-multiple-traces

# # https://plotly.com/python/discrete-color/
# fig = px.colors.qualitative.swatches()

# #https://plotly.com/python/renderers/
# fig.show(renderer='browser')

In [31]:
custom_colors = [['#F44336', '#FF8A80', '#FF5252', '#FF1744', '#D50000'], 
                 ['#E91E63', '#FF80AB', '#FF4081', '#F50057', '#C51162'],
                 ['#9C2780', '#EA80FC', '#E040FB', '#D500F9', '#AA00FF'],
                 ['#673AB7', '#B388FF', '#7C4DFF', '#651FFF', '#6200EA'],
                 ['#3F51B5', '#8C9EFF', '#536DFE', '#3D5AFE', '#304FFE'],
                 
                 ['#2196F3', '#82B1FF', '#448AFF', '#2979FF', '#2962FF'],
                 ['#03A9F4', '#80D8FF', '#40C4FF', '#00B0FF', '#0091EA'],
                 ['#00BCD4', '#84FFFF', '#18FFFF', '#00E5FF', '#00B8D4'],
                 ['#009688', '#A7FFEB', '#64FFDA', '#1DE9B6', '#00BFA5'],
                 ['#4CAF50', '#B9F6CA', '#69F0AE', '#00E676', '#00C853'],
                 
                 ['#8BC34A', '#CCFF90', '#B2FF59', '#76FF03', '#64DD17'],
                 ['#CDDC39', '#F4FF81', '#EEFF41', '#C6FF00', '#AEEA00'],
                 ['#FFEB3B', '#FFFF8D', '#FFFF00', '#C6FF00', '#FFD600'],
                 ['#FFC107', '#FF9E80', '#FF6E40', '#FF3D00', '#DD2C00'],
                 ['#FF9800', '#FFD180', '#FFAB40', '#FF9100', '#FF6D00'],
                 
                 ['#FF5722', '#FF9E80', '#FF6E40', '#FF3D00', '#DD2C00'],
                 ['#D7CCC8', '#A1887F', '#795548', '#3E2723', '#07090F'],
                 ['#9E9E9E', '#EEEEEE', '#BDBDBD', '#757575', '#424242'],
                 ['#607D8B', '#B0BEC5', '#78909C', '#546E7A', '#37474F']]

In [32]:

# custom_colors1 = []
# for color in custom_colors:
#     custom_colors1.append(color[0]) 
# px.colors.qualitative.custom_colors1 = custom_colors1

In [33]:
# custom_colors2 = []
# for color in custom_colors:
#     custom_colors2.append(color[1]) 
# px.colors.qualitative.custom_colors2 = custom_colors2

In [34]:
# custom_colors3 = []
# for color in custom_colors:
#     custom_colors3.append(color[2]) 
# px.colors.qualitative.custom_colors3 = custom_colors3

In [35]:
# custom_colors4 = []
# for color in custom_colors:
#     custom_colors4.append(color[3]) 
# px.colors.qualitative.custom_colors4 = custom_colors4

In [36]:
# custom_colors5 = []
# for color in custom_colors:
#     custom_colors5.append(color[4]) 
# px.colors.qualitative.custom_colors5 = custom_colors5

In [37]:
# custom_colors = np.array(custom_colors)
# custom_colors = custom_colors.reshape(95)
# custom_colors = custom_colors.tolist()
# px.colors.qualitative.custom_colors = custom_colors
# fig = px.colors.qualitative.swatches()
# fig.show(renderer='browser')

In [38]:
# # make easy to paste activity variables
# for activity in unique_activities:
#     print(f"{activity.lower().replace('/', '_').replace(' ', '_')} = ")

In [39]:
working = '#D50000'              #custom_colors[0][3]
sleeping = '#2196F3'             #custom_colors[6][0]
grooming = '#80D8FF'             #custom_colors[8][0]
internet = '#00C853'             #custom_colors[10][3]
movies_tv = '#00C853'
programming = '#00E676'
games = '#B9F6CA'
gamedev = '#00E676'

walking = '#BDBDBD'
shopping = '#32514B'
eating = '#18FFFF'
cooking = '#00BFA5'
cleaning = '#1DE9B6'
dishes = '#64FFDA'

driving = '#757575'
gym = '#9C27B0'
masturbation = '#FF80AB'         #custom_colors[7][1]
sex = '#FF80AB' 
personal_time = '#FF80AB' 
toilet = '#FFFF8D'               #custom_colors[13][1]

classes = '#BCA68E' 
drinking = '#00B8D4'
phone = '#D7CCC8'
drumming = '#8C9EEE'
writing = '#536DFE'
reading = '#3D5AFE'
keyboard = '#E040FB'
drawing = '#AA00FF'
swimming = '#80D8FF'
biking = '#651FFF'
mycology = '#EA80FC'

studying = '#BCA68E' 
errands = '#A1887F'
appointments = '#A1887F'
data_analysis = '#78909C'
dating = '#FF80AB'
family_time = '#FF9100'
friends_time = '#FFAB40'
event = '#AA00FF'
dancing = '#D500F9'
club = '#FFD180'

In [40]:
# https://stackoverflow.com/questions/63460213/plotly-how-to-define-colors-in-a-figure-using-plotly-graph-objects-and-plotly-e
# https://community.plotly.com/t/different-colors-for-bars-in-barchart-by-their-value/6527/7
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph

colors = {'Working': f'{working}',
          'Sleeping': f'{sleeping}',
          'Grooming': f'{grooming}',
          'Internet': f'{internet}',
          'Movies/TV': f'{movies_tv}',
          'Programming': f'{programming}',
          'Games': f'{games}',
          'GameDev': f'{gamedev}',
          
          'Walking': f'{walking}',
          'Shopping': f'{shopping}',
          'Eating': f'{eating}',
          'Cooking': f'{cooking}',
          'Cleaning': f'{cleaning}',
          'Dishes': f'{dishes}',
          
          'Driving': f'{driving}',
          'Gym': f'{gym}',
          'Masturbation': f'{masturbation}',
          'Sex': f'{sex}',
          'Personal Time': f'{personal_time}',
          'Toilet': f'{toilet}',
          
          'Classes': f'{classes}',
          'Drinking': f'{drinking}',
          'Phone': f'{phone}',
          'Drumming': f'{drumming}',
          'Writing': f'{writing}',
          'Reading': f'{reading}',
          'Keyboard': f'{keyboard}',
          'Drawing': f'{drawing}',
          'Swimming/Tanning': f'{swimming}',
          'Biking': f'{biking}',
          'Mycology': f'{mycology}',
          
          'Studying': f'{studying}',
          'Errands': f'{errands}',
          'Appointments': f'{appointments}',
          'Errands and Appointments': f'{errands}',
          'Data Analysis': f'{data_analysis}',
          'Dating': f'{dating}',
          'Family Time': f'{family_time}',
          'Friends Time': f'{friends_time}',
          'Event': f'{event}',
          'Dancing': f'{dancing}',
          'Club/Social Group': f'{club}',
         }

colors

# kinda lost the plot on what this was for, don't need it apparently.
# marker_colors = {'Colors': {activity:color for i in unique_activities for activity, color in colors.items()}}
# marker_colors

{'Working': '#D50000',
 'Sleeping': '#2196F3',
 'Grooming': '#80D8FF',
 'Internet': '#00C853',
 'Movies/TV': '#00C853',
 'Programming': '#00E676',
 'Games': '#B9F6CA',
 'GameDev': '#00E676',
 'Walking': '#BDBDBD',
 'Shopping': '#32514B',
 'Eating': '#18FFFF',
 'Cooking': '#00BFA5',
 'Cleaning': '#1DE9B6',
 'Dishes': '#64FFDA',
 'Driving': '#757575',
 'Gym': '#9C27B0',
 'Masturbation': '#FF80AB',
 'Sex': '#FF80AB',
 'Personal Time': '#FF80AB',
 'Toilet': '#FFFF8D',
 'Classes': '#BCA68E',
 'Drinking': '#00B8D4',
 'Phone': '#D7CCC8',
 'Drumming': '#8C9EEE',
 'Writing': '#536DFE',
 'Reading': '#3D5AFE',
 'Keyboard': '#E040FB',
 'Drawing': '#AA00FF',
 'Swimming/Tanning': '#80D8FF',
 'Biking': '#651FFF',
 'Mycology': '#EA80FC',
 'Studying': '#BCA68E',
 'Errands': '#A1887F',
 'Appointments': '#A1887F',
 'Errands and Appointments': '#A1887F',
 'Data Analysis': '#78909C',
 'Dating': '#FF80AB',
 'Family Time': '#FF9100',
 'Friends Time': '#FFAB40',
 'Event': '#AA00FF',
 'Dancing': '#D500F9',
 'C

In [41]:
# df['Colors'] = df['Activity'].map(colors)
# df

In [42]:
# # alternative for if you want custom colors for certain words in the tags ***or based on groups***
# # https://stackoverflow.com/questions/49161120/pandas-python-set-value-of-one-column-based-on-value-in-another-column
# # https://stackoverflow.com/questions/23400743/pandas-modify-column-values-in-place-based-on-boolean-array
# #px.colors.qualitative.Plotly

# custom_colors = ['#EF553B'] * 200
# df['colors'] = custom_colors

# activity = 'Sleeping'
# df.loc[df['tags'].str.contains(sleeping), 'Colors'] = '#636EFA'
# #df.loc[df['tags'].str.contains(sleeping), 'Colors'] = '#636EFA'

# activity = 'Working'
# df.loc[df['tags'].str.contains(activity), 'Colors'] = '#00CC96'

# activity = 'Internet'
# df.loc[df['tags'].str.contains(activity), 'Colors'] = '#AB63FA'

# df

# Start of getting rolling starts/ends, sum, etc, of activities

In [43]:
# # original of average_sleep_df
# # https://stackoverflow.com/questions/52007139/get-the-average-date-from-multiple-dates-pandas

# average_sleep_df = pd.DataFrame()
# average_sleep_df['Start'] = [x.value for x in df['Start']]
# average_sleep_df['End'] = [x.value for x in df['End']]

# average_sleep_df.describe()

In [44]:
# # https://stackoverflow.com/questions/52007139/get-the-average-date-from-multiple-dates-pandas

# activity = 'Sleeping'

# average_sleep_df = pd.DataFrame()
# average_sleep_df['Start'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(activity), 'Start'])]
# average_sleep_df['End'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(activity), 'End'])]

# average_sleep_df.describe()

In [45]:
# fig = px.scatter(average_sleep_df)
# fig.show(renderer='browser')

In [46]:
# # https://stackoverflow.com/questions/19231871/convert-unix-time-to-readable-date-in-pandas-dataframe
# mean_sleep_start = pd.to_datetime(df.loc[df['Activity'].str.contains(activity), 'Start'].mean())
# mean_sleep_start

In [47]:
# median_sleep_start = pd.to_datetime(df.loc[df['Activity'].str.contains(activity), 'Start'].median())
# median_sleep_start

In [48]:
# mean_sleep_end = pd.to_datetime(df.loc[df['Activity'].str.contains(activity), 'End'].mean())
# mean_sleep_end

In [49]:
# median_sleep_end = pd.to_datetime(df.loc[df['Activity'].str.contains(activity), 'End'].median())
# median_sleep_end

In [50]:
# sum_sleep_durations = df.loc[df['Activity'].str.contains(activity), 'Duration in Seconds'].sum()
# sum_sleep_durations

In [51]:
# sum_sleep_durations / 60 / 60 # in hours

In [52]:
# # converting sum_<activity>_ durations into total 'D:H:M:S'
# sleep_timedelta = str(pd.Timedelta(seconds=sum_sleep_durations))[-8:]
# sleep_timedelta

In [53]:
# sleep_daydelta = str(pd.Timedelta(seconds=sum_sleep_durations).days)
# sleep_daydelta

In [54]:
# sleep_daydelta + ':' + sleep_timedelta

In [55]:
# 2984.966388888889 / 8760

In [56]:
# std_sleep_durations = df.loc[df['Activity'].str.contains(activity), 'Duration in Seconds'].std()
# std_sleep_durations / 60 / 60 

In [57]:
# sum_gym_durations = df.loc[df['Activity'].str.contains('Gym'), 'Duration in Seconds'].sum()
# sum_gym_durations / 60

In [58]:
# mean_sleep_duration = df.loc[df['Activity'].str.contains('Sleeping'), 'Duration in Seconds'].mean()
# mean_sleep_duration / 60 

In [59]:
# median_sleep_duration = df.loc[df['Activity'].str.contains('Sleeping'), 'Duration in Seconds'].median()
# median_sleep_duration / 60 

In [60]:
# std_sleep_duration = df.loc[df['Activity'].str.contains('Sleeping'), 'Duration in Seconds'].std()
# std_sleep_duration / 60 

In [61]:
# for activity in unique_activities:
#     print(activity.lower().replace('/', '_').replace(' ', '_'))

In [62]:
activities = ['Internet', 'Gym', 'Personal Time']

In [63]:
# total seconds in day recorded for all activities
# can get % of day from this for each activity
pd.DataFrame(df.groupby(df['Day']).agg('Duration in Seconds').sum()).reset_index()

,Day,Duration in Seconds
0,2020-12-31,1758.0
1,2021-01-01,90256.0
2,2021-01-02,99009.0
3,2021-01-03,108802.0
4,2021-01-04,108501.0
...,...,...
362,2021-12-28,86519.0
363,2021-12-29,86558.0
364,2021-12-30,86933.0
365,2021-12-31,87208.0


In [64]:
# duration in seconds of the activity
pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').sum()).reset_index()

NameError: name 'activity' is not defined

In [65]:
# counts of the days of activity
pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').count()).reset_index().rename({'Duration in Seconds':'Counts'}, axis='columns')

NameError: name 'activity' is not defined

In [66]:
def lower_replace(string):
    return string.lower().replace('/', '_').replace(' ', '_')

In [67]:
lower_replace('Internet Stuff')

'internet_stuff'

In [68]:
# to get df's for heatmap, add to ActivityStats after getting it working
merged_df_list = []
for activity in unique_activities_sorted:
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Day']
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] =  df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']       
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] =  [pd.to_datetime(i) for i in globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day']]
    globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] = [int(i) for i in globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')]]
    
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').sum()).reset_index().rename({'Duration in Seconds':f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')}, axis='columns')
    #print(f"{activity}:", globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].columns) # debugging when the names change
    globals()[f"grouped_{activity}_count_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame(df.loc[df['Activity'].str.contains(f"{activity}")].groupby(df['Day']).agg('Duration in Seconds').count()).reset_index().rename({'Duration in Seconds':f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')}, axis='columns')
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_count_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_counts".lower().replace('/', '_').replace(' ', '_')]
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] =  [pd.to_datetime(i) for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day']]
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')] = [int(i) for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')]]

    #print(type(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')])) # debugging when dataframe changes
    
    # add percentage of counts of activity / total counts activities in day as another column? maybe others to correlate as well

#     end = pd.to_datetime(df['Day'].iloc[-1])
#     start = pd.to_datetime(df['Day'][0])
    
    # https://www.kite.com/python/answers/how-to-insert-a-row-into-a-pandas-dataframe
    # https://stackoverflow.com/questions/19324453/add-missing-dates-to-pandas-dataframe
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.asfreq.html#pandas.DataFrame.asfreq
#     row_df_start = pd.DataFrame({'Day':[start],
#                           'Duration in Seconds' : 0,
#                           #'Counts' : 0
#                           })
#     row_df_end = pd.DataFrame({'Day':[end],
#                           'Duration in Seconds' : 0,
#                           #'Counts' : 0
#                           })

#     globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.concat([row_df_start, globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]], ignore_index=True)
#     globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.concat([globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_'), row_df_end]], ignore_index=True)

    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].set_index('Day')
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].asfreq('D', fill_value=0).reset_index()
#     globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].drop([0, 1]).reset_index()
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = 0
    globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds_7_day_rolling_mean".lower().replace('/', '_').replace(' ', '_')] = 0

    for i in globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')].index:
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations".lower().replace('/', '_').replace(' ', '_')][i] = str(pd.Timedelta(seconds=globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')][i]))[-8:]
        globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds_7_day_rolling_mean".lower().replace('/', '_').replace(' ', '_')][i] = np.nan_to_num(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')][f"{activity}_durations_seconds".lower().replace('/', '_').replace(' ', '_')].rolling(7).mean()[i])

    merged_df_list.append(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')])
    
    # there is a more efficient way for the above loop with .loc or .apply but idk how to get it to work
    # https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
    #grouped_programming_df.loc[grouped_programming_df['Seconds'], ('Duration in Seconds', i)] = str(pd.Timedelta(seconds=grouped_programming_df['Seconds']))[-8:]    

#     del globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['index']

# need either to trim it to single year, or leave be and make the trimming in another function/the heatmap builder
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.truncate.html
grouped_internet_df
# should look into pd.df.interpolate https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html

NameError: name 'unique_activities_sorted' is not defined

In [69]:
grouped_internet_df.dtypes

NameError: name 'grouped_internet_df' is not defined

In [70]:
[i.head(2) for i in merged_df_list]

[]

In [71]:
# merged_df_list = [', '.join(merge_df_list)]
# merged_df_list = [i.replace("'", "") for i in merged_df_list]
# merged_df_list

In [72]:
# merged_df_list = []
# for activity in unique_activities_sorted:
#     merged_df_list.merge([i for i in activity_correlations_df], on='Day')

In [73]:
# # example of merging df's
# temp_df = pd.DataFrame()
# temp_df = grouped_internet_df.merge(grouped_sleeping_df, on='Day')
# temp_df

In [74]:
# how to merge the df's into one to make a correlation matrix
# https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
from functools import reduce

In [75]:
activity_correlations_df = reduce(lambda  left, right: pd.merge(left, right, on=['Day'], how='outer'), merged_df_list).fillna(0)
activity_correlations_df

TypeError: reduce() of empty sequence with no initial value

In [76]:
# activity_correlations_df = pd.DataFrame()
# for activity in unique_activities_sorted:
#     activity_correlations_df.merge(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')], on='Day')
# activity_correlations_df    

In [77]:
# # show head of each of those created grouped_activity_df's
# for activity in unique_activities_sorted:
#     print(f"{activity}", globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].shape)
#     print(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].head(3))
#     print(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].tail(3))
#     print()

In [78]:
# with pd.option_context('display.max_rows', 400, 'display.max_columns', 5):
#       display(grouped_personal_time_df)

In [79]:
# print(grouped_internet_df['Day'][0])
# print(type(grouped_internet_df['Day'][0]))
# print()

# print(grouped_internet_df['internet_durations_seconds'][0])
# print(type(grouped_internet_df['internet_durations_seconds'][0]))
# print()

# print(grouped_internet_df['internet_counts'][0])
# print(type(grouped_internet_df['internet_counts'][0]))
# print()

# print(grouped_internet_df['internet_durations'][0])
# print(type(grouped_internet_df['internet_durations'][0]))
# print()

In [80]:
# export 
activity_correlations_df.to_csv('data/activity_correlations_df.csv', index=None, encoding='utf-8')

NameError: name 'activity_correlations_df' is not defined

# Correlation matrix

In [81]:
# do this for the general grouped_activity_df, that you haven't created yet that will contain the 
# day, duration in seconds, counts, of each activity in a single df
# correlations between each series against eachother

# time delayed/lagged correlation of activity with itself in the past/future
# https://pandas.pydata.org/docs/reference/api/pandas.Series.autocorr.html
# https://www.influxdata.com/blog/autocorrelation-in-time-series-data/
# https://www.freecodecamp.org/news/how-machines-make-predictions-finding-correlations-in-complex-data-dfd9f0d87889/
# https://towardsdatascience.com/advanced-time-series-analysis-in-python-decomposition-autocorrelation-115aa64f475e

# get rolling window correlations, 3 day, 7, 14, 28, etc.

# arima 
# https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html

# linear regression
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

# https://towardsdatascience.com/four-ways-to-quantify-synchrony-between-time-series-data-b99136c4a9c9

activity_correlations_df.corr(method='pearson')
#grouped_internet_df.corrwith(grouped_grooming_df, method='pearson')

# use corrwith
# for activity in unique_activities_sorted:
#     print(f"{activity}:", round(globals()[f"grouped_{activity}_df".lower().replace('/', '_').replace(' ', '_')].corr(method='pearson')['Duration in Seconds'][1], 3))

NameError: name 'activity_correlations_df' is not defined

In [82]:
from scipy.stats import pearsonr
import numpy as np
rho = activity_correlations_df.corr().round(3)
pval = activity_correlations_df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.001, 0.01, 0.05] if x<=t])) # can add more * for even smaller p value's e.g. 0.00001, 0.0001, in the list 
correlation_matrix = rho.astype(str) + p
correlation_matrix

NameError: name 'activity_correlations_df' is not defined

In [ ]:
with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
    display(correlation_matrix)

In [83]:
# with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
#     display(correlation_matrix['personal_time_durations_seconds_7_day_rolling_mean'])

# ActivityStats class

In [84]:
# https://www.codegrepper.com/code-examples/python/how+to+create+dynamic+variable+names+in+python

# make rolling averages, only will work if all the average_activity_df's have the same day
# make time since last instance
# https://stackoverflow.com/questions/51856844/python-pandas-minutes-since-last-occurrence-in-2-million-row-dataframe
# add longest streak of activities, i.e. 365 days in a row on internet or 7 days in a row gaming, etc.

# make .corr matrix of instances per day/durations per day and .cov (covariance)
# make data table
# make heatmap of per activity instance in day and of total amount of durations in day
# make histograms of each activity's durations per day vs instances at time those durations, bin = minute?
# make violin/ridgeplots for each of the activities as a y, https://plotly.com/python/violin/
# make normal distribution histogram, mean/median = middle, std = lines
# show "Created _df, sum_, mean_, median_, etc."

# gmean, hmean, zscores
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gmean.html#scipy.stats.gmean
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.hmean.html
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html
# https://www.simplypsychology.org/z-score.html
# range, absolute minimum vs calculated minimum, i.e. 0 seconds vs of values above 0
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.iqr.html 75th q - 25th q, gets rid of outliers

# make interpolated counts df, for each activity, then mean/median/sum/quantile/min/max df of all of the counts of activities

# make the splitting of the df's at midnight a part of this class __init__ to get accurate activity_df['Start']/['End']'s, without the 1970 though

class ActivityStats:
    def __init__(self, df=df):
        self.df = df
        self.unique_activities = self.df['Activity'].unique().tolist()
        self.days_in_df = pd.DataFrame(self.df['Day'].unique(), columns=['Day'])
        self.num_days_in_df = len(self.days_in_df) - 1
        self.seconds_in_df = self.num_days_in_df * 86400
        
        globals()["unique_activities"] = self.unique_activities
        globals()["days_in_df"] = self.days_in_df
        globals()["num_days_in_df"] = self.num_days_in_df
        globals()["seconds_in_df"] = self.seconds_in_df
                
        for activity in self.unique_activities:
            globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
            globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Start'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'Start'])]
            globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['End'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'End'])]
            
#             globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Start'].mean()))
#             globals()[f"median_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Start'].median()))
#             globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['End'].mean()))
#             globals()[f"median_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(round(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['End'].median()))
            
            # start of activity df's for heatmap from above, i.e. insert whatever you make above into here
            globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
            globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Day'] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Day']
            globals()[f"{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Duration in Seconds'] =  df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']            
            
            
            
            
            
            
# percent of activities of total activity per day, i.e., 3 instances of internet in day, 24 activities, 3/24 = 12.5% of activities in day was internet.
# percent of days that I engaged in that activity, i.e. I engaged in internet 365 days in year == 100%

            globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'Start'].mean())
            globals()[f"median_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'Start'].median())
            globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'End'].mean())
            globals()[f"median_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'End'].median())

            globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].sum()
            globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).mean()
            globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).median()
            globals()[f"std_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = np.nan_to_num(round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).std())
            globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].min()
            globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].max()

            globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] = len(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')])
            globals()[f"mean_num_{activity}_per_day".lower().replace('/', '_').replace(' ', '_')] = round(globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] / self.num_days_in_df, 2)
            globals()[f"{activity}_percent".lower().replace('/', '_').replace(' ', '_')] = (globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] / self.seconds_in_df) * 100
    
            globals()[f"sum_{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]))[-8:]
            globals()[f"sum_{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]).days)
            globals()[f"sum_{activity}_dhms".lower().replace('/', '_').replace(' ', '_')] = globals()[f"sum_{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] + ':' + globals()[f"sum_{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')]
            globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"std_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"std_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            
            globals()[f"25_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).quantile(.25)
            globals()[f"75_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).quantile(.75)
            globals()[f"25_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"25_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            globals()[f"75_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"75_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
            
        total_seconds_list = []
        for activity in unique_activities:
            total_seconds_list.append([f"{activity}", globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]])
        total_seconds_list = sorted(total_seconds_list, key=lambda x: x[1], reverse=True)
        unique_activities_sorted = sorted(total_seconds_list, key=lambda x: x[1], reverse=True)
        self.unique_activities_sorted = [i[0] for i in unique_activities_sorted]
        globals()["unique_activities_sorted"] = self.unique_activities_sorted
   
    def activities(self):
        return [activity for activity in self.unique_activities]

    def min_durations(self):
        globals()['min_activity_durations_hms'] = pd.DataFrame()
        globals()['min_activity_durations_hms']['Activity'] = [i for i in self.unique_activities_sorted]
        globals()['min_activity_durations_hms']['min_activity_durations_hms'] = [globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        print("Created 'min_activity_durations_hms'")
        return globals()['min_activity_durations_hms']

    def max_durations(self):
        globals()['max_activity_durations_hms'] = pd.DataFrame()
        globals()['max_activity_durations_hms']['Activity'] = [i for i in self.unique_activities_sorted]
        globals()['max_activity_durations_hms']['max_activity_durations_hms'] = [globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        print("Created 'max_activity_durations_hms'")
        return globals()['max_activity_durations_hms']     

    def mean_std_durations(self):
        globals()['mean_std_durations'] = pd.DataFrame()
        globals()['mean_std_durations']['Activity'] = [i for i in self.unique_activities_sorted]
        globals()['mean_std_durations']['mean_std_durations'] = [globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] + 
                                                                 ' \u00B1 ' + 
                                                                 globals()[f"std_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')]
                                                                 for activity in self.unique_activities_sorted]
        print("Created 'mean_std_durations'")
        return globals()['mean_std_durations']

    # make median_std_durations like above
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.median_abs_deviation.html#scipy.stats.median_abs_deviation
    #https://stackoverflow.com/questions/61436389/use-custom-function-median-absolute-deviation-in-pandas-transform
    #https://stackoverflow.com/questions/22804036/median-absolute-deviation-mad-based-outlier-detection
    #https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.median_absolute_deviation.html
    
    # make percentile_durations with required argument of int(0-100)
    # np.percentile, pd.percentile
    
    def describe(self, ):
        
        # activity_df
        globals()['activity_df'] = pd.DataFrame()
        globals()['activity_df_seconds'] = pd.DataFrame()
#         globals()['activity_df_times'] = pd.DataFrame()

        globals()['activity_df']['Activity'] = [activity for activity in self.unique_activities_sorted]
        globals()['activity_df']['total_num_instance_activity'] = [globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['mean_num_activity_per_day'] = [globals()[f"mean_num_{activity}_per_day".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['activity_percent'] = [globals()[f"{activity}_percent".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

        globals()['activity_df']['mean_activity_start'] = [globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['median_activity_start'] = [globals()[f"median_{activity}_start".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['mean_activity_end'] = [globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['median_activity_end'] = [globals()[f"median_{activity}_end".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
# rolling averages instances etc
        
        globals()['activity_df_seconds']['Activity'] = [activity for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['sum_activity_durations_seconds'] = [globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['mean_activity_durations_seconds'] = [globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['std_activity_durations_seconds'] = [globals()[f"std_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['min_activity_durations_seconds'] = [globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['25_quant_activity_durations'] = [globals()[f"25_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['median_activity_durations_seconds'] = [globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['75_quant_activity_durations'] = [globals()[f"75_quant_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df_seconds']['max_activity_durations_seconds'] = [globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

#         globals()['activity_df']['sum_activity_timedelta'] = [globals()[f"sum_{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
#         globals()['activity_df']['sum_activity_daydelta'] = [globals()[f"sum_{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

        globals()['activity_df']['sum_activity_dhms'] = [globals()[f"sum_{activity}_dhms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['mean_activity_durations_hms'] = [globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['std_activity_durations_hms'] = [globals()[f"std_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['min_activity_durations_hms'] = [globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]       
        globals()['activity_df']['25_quant_activity_durations_hms'] = [globals()[f"25_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['median_activity_durations_hms'] = [globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['75_quant_activity_durations_hms'] = [globals()[f"75_quant_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]
        globals()['activity_df']['max_activity_durations_hms'] = [globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] for activity in self.unique_activities_sorted]

        globals()['activity_df_transposed'] = globals()['activity_df'].set_index('Activity').T
        globals()['activity_df_transposed'] = globals()['activity_df_transposed'].reset_index()
        globals()['activity_df_seconds_transposed'] = globals()['activity_df_seconds'].set_index('Activity').T
        globals()['activity_df_seconds_transposed'] = globals()['activity_df_seconds_transposed'].reset_index()

        # activity_df_totals
        globals()['activity_df_totals'] = pd.DataFrame()
        globals()['activity_df_totals']['total_durations_seconds_in_df'] = [globals()['activity_df_seconds']['sum_activity_durations_seconds'].sum()]        
        globals()[f"total_durations_timedelta"] = str(pd.Timedelta(seconds=globals()['activity_df_totals']['total_durations_seconds_in_df'][0]))[-8:]
        globals()[f"total_durations_daydelta"] = str(pd.Timedelta(seconds=globals()['activity_df_totals']['total_durations_seconds_in_df'][0]).days)
        globals()['activity_df_totals']['total_durations_dhms_in_df'] = globals()[f"total_durations_daydelta"] + ':' + globals()[f"total_durations_timedelta"]
        globals()['activity_df_totals']['total_days_in_df'] = self.num_days_in_df
        globals()['activity_df_totals']['num_unique_activities_in_df'] = len(self.unique_activities)
        globals()['activity_df_totals']['total_instances_of_activities_in_df'] = [globals()['activity_df']['total_num_instance_activity'].sum()]
        
        self.total_activities_per_day_df = pd.DataFrame(df.groupby(df['Day']).agg('Duration').count().reset_index()).rename({'Duration':'Counts'}, axis='columns')
        globals()["total_activities_per_day_df"] = self.total_activities_per_day_df
        
        self.total_activities_per_day_mean = round(self.total_activities_per_day_df['Counts'].mean(), 2)
        self.total_activities_per_day_std = round(self.total_activities_per_day_df['Counts'].std(), 2)
        self.total_activities_per_day_min = int(self.total_activities_per_day_df['Counts'].min())
        self.total_activities_per_day_25 = int(self.total_activities_per_day_df['Counts'].quantile(.25))
        self.total_activities_per_day_median = int(self.total_activities_per_day_df['Counts'].median())
        self.total_activities_per_day_75 = int(self.total_activities_per_day_df['Counts'].quantile(.75))
        self.total_activities_per_day_max = int(self.total_activities_per_day_df['Counts'].max())
        
        globals()['activity_df_totals']['total_activities_per_day_mean'] = [self.total_activities_per_day_mean]
        globals()['activity_df_totals']['total_activities_per_day_std'] = [self.total_activities_per_day_std]
        globals()['activity_df_totals']['total_activities_per_day_min'] = [self.total_activities_per_day_min]
        globals()['activity_df_totals']['total_activities_per_day_25'] = [self.total_activities_per_day_25]
        globals()['activity_df_totals']['total_activities_per_day_median'] = [self.total_activities_per_day_median]
        globals()['activity_df_totals']['total_activities_per_day_75'] = [self.total_activities_per_day_75]
        globals()['activity_df_totals']['total_activities_per_day_max'] = [self.total_activities_per_day_max]
        
#         globals()['total_activities_per_day_mean'] = self.activities_per_day_mean
#         globals()['total_activities_per_day_std'] = self.activities_per_day_std
#         globals()['total_activities_per_day_25'] = self.activities_per_day_25
#         globals()['total_activities_per_day_median'] = self.activities_per_day_median
#         globals()['total_activities_per_day_75'] = self.activities_per_day_75
#         globals()['total_activities_per_day_max'] = self.activities_per_day_max

        
        globals()['activity_df_totals'] = globals()['activity_df_totals'].T
        globals()['activity_df_totals'] = globals()['activity_df_totals'].rename(columns={0:'activity_df_totals'})
        globals()['activity_df_totals'] = globals()['activity_df_totals'].reset_index()
        #globals()['activity_df_totals'] = globals()['activity_df_totals'].set_index('index')
        
        print("Created 'activity_df,' 'activity_df_transposed,'")
        print("creaed 'activity_df_seconds', 'activity_df_seconds_transposed,'")
        print("Created 'activity_df_totals,'")
        return globals()['activity_df_transposed']

In [85]:
# for activity in unique_activities:
#     print(f'{activity}\n', globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')].head(2))

In [86]:
ActivityStats(df=original_df).describe()

Created 'activity_df,' 'activity_df_transposed,'
creaed 'activity_df_seconds', 'activity_df_seconds_transposed,'
Created 'activity_df_totals,'


Activity,index,Sleeping,Working,Internet,Programming,Games,Family Time,Eating,Grooming,Friends Time,...,Writing,Errands and Appointments,Drawing,Concert/Event,Phone,Drinking,Swimming/Tanning,Biking,Drumming,Dancing
0,total_num_instance_activity,427,173,1269,324,103,70,1144,730,30,...,20,38,3,1,5,10,2,1,3,1
1,mean_num_activity_per_day,1.17,0.47,3.48,0.89,0.28,0.19,3.13,2.0,0.08,...,0.05,0.1,0.01,0.0,0.01,0.03,0.01,0.0,0.01,0.0
2,activity_percent,33.921185,22.881795,20.877087,4.202873,3.392431,2.977584,2.872495,2.325637,2.170475,...,0.147435,0.139675,0.078142,0.039361,0.036466,0.033489,0.02949,0.02109,0.016261,0.003006
3,mean_activity_start,2021-07-05 19:09:27.250585600,2021-06-24 19:59:24.231214080,2021-07-05 07:48:34.877068544,2021-06-04 17:59:36.706790144,2021-06-20 19:11:05.339805696,2021-07-15 19:10:11.014286080,2021-07-13 01:50:53.625874176,2021-07-13 17:59:53.875342336,2021-05-27 16:32:57.366666496,...,2021-01-22 06:59:35.949999872,2021-06-19 11:27:07.973684224,2021-01-29 09:33:29.333333248,2021-07-24 19:54:24,2021-09-07 00:02:28.399999744,2021-07-26 01:14:00.800000,2021-07-03 15:33:30,2021-10-05 13:57:33,2021-08-31 08:19:06.999999744,2021-07-24 23:24:14
4,median_activity_start,2021-07-04 09:19:43,2021-06-16 00:00:00,2021-07-06 16:37:01,2021-06-07 02:47:02.500000,2021-07-29 00:14:46,2021-07-13 03:49:01,2021-07-18 13:17:52.500000,2021-07-16 12:25:30.500000,2021-05-26 15:30:35.500000,...,2021-01-22 02:43:49.500000,2021-06-21 12:37:35.500000,2021-01-29 02:35:10,2021-07-24 19:54:24,2021-09-07 13:45:00,2021-07-16 22:02:57.500000,2021-07-03 15:33:30,2021-10-05 13:57:33,2021-11-16 15:15:33,2021-07-24 23:24:14
5,mean_activity_end,2021-07-06 01:25:44.936768256,2021-06-25 03:46:43.156069632,2021-07-05 09:08:55.982663424,2021-06-04 19:02:17.555555584,2021-06-20 21:47:43.058253056,2021-07-15 22:53:45.457142784,2021-07-13 02:04:03.863636480,2021-07-13 18:16:38.501369856,2021-05-27 21:28:31.700000,...,2021-01-22 07:38:20.699999744,2021-06-19 11:46:27.131578880,2021-01-29 11:50:23.666666752,2021-07-24 23:21:17,2021-09-07 00:35:17.600000,2021-07-26 01:30:14.600000,2021-07-03 16:51:00,2021-10-05 15:48:24,2021-08-31 08:47:36.333333248,2021-07-24 23:40:02
6,median_activity_end,2021-07-04 14:10:08,2021-06-16 07:48:02,2021-07-06 18:50:01,2021-06-07 03:54:20,2021-07-29 02:25:35,2021-07-13 04:14:21.500000,2021-07-18 13:33:57,2021-07-16 12:44:41.500000,2021-05-26 22:39:55,...,2021-01-22 03:18:50,2021-06-21 12:40:37,2021-01-29 05:45:25,2021-07-24 23:21:17,2021-09-07 14:12:00,2021-07-16 22:20:28.500000,2021-07-03 16:51:00,2021-10-05 15:48:24,2021-11-16 15:45:03,2021-07-24 23:40:02
7,sum_activity_dhms,123:19:29:45,83:12:26:43,76:04:49:58,15:08:10:18,12:09:10:37,10:20:50:11,10:11:37:50,8:11:43:33,7:22:08:01,...,0:12:54:55,0:12:14:08,0:06:50:43,0:03:26:53,0:03:11:40,0:02:56:01,0:02:35:00,0:01:50:51,0:01:25:28,0:00:15:48
8,mean_activity_durations_hms,06:57:32,11:35:11,01:26:28,01:08:11,02:53:07,03:43:34,00:13:12,00:16:45,06:20:16,...,00:38:45,00:19:19,02:16:54,03:26:53,00:38:20,00:17:36,01:17:30,01:50:51,00:28:29,00:15:48
9,std_activity_durations_hms,02:52:39,01:57:38,01:17:42,00:43:53,01:45:20,02:01:28,00:08:28,00:17:41,03:39:46,...,00:22:41,00:21:37,00:49:41,00:00:00,00:32:15,00:11:29,00:09:12,00:00:00,00:02:35,00:00:00


In [87]:
# with pd.option_context('display.max_rows', 50, 'display.max_columns', 50):
#     display(activity_df_transposed)

In [88]:
activity_df_totals

,index,activity_df_totals
0,total_durations_seconds_in_df,34526911.0
1,total_durations_dhms_in_df,399:14:48:31
2,total_days_in_df,365
3,num_unique_activities_in_df,33
4,total_instances_of_activities_in_df,9436
5,total_activities_per_day_mean,26.85
6,total_activities_per_day_std,7.14
7,total_activities_per_day_min,1
8,total_activities_per_day_25,22
9,total_activities_per_day_median,27


In [89]:
ActivityStats(df=original_df).mean_std_durations()

Created 'mean_std_durations'


,Activity,mean_std_durations
0,Sleeping,06:57:32 ± 02:52:39
1,Working,11:35:11 ± 01:57:38
2,Internet,01:26:28 ± 01:17:42
3,Programming,01:08:11 ± 00:43:53
4,Games,02:53:07 ± 01:45:20
5,Family Time,03:43:34 ± 02:01:28
6,Eating,00:13:12 ± 00:08:28
7,Grooming,00:16:45 ± 00:17:41
8,Friends Time,06:20:16 ± 03:39:46
9,Gym,01:02:21 ± 00:06:28


In [90]:
unique_activities_sorted

['Sleeping',
 'Working',
 'Internet',
 'Programming',
 'Games',
 'Family Time',
 'Eating',
 'Grooming',
 'Friends Time',
 'Gym',
 'Reading',
 'Movies/TV',
 'Toilet',
 'Studying',
 'Driving',
 'Walking',
 'Class',
 'Dating',
 'Cooking',
 'Dishes',
 'Personal Time',
 'Cleaning',
 'Shopping',
 'Writing',
 'Errands and Appointments',
 'Drawing',
 'Concert/Event',
 'Phone',
 'Drinking',
 'Swimming/Tanning',
 'Biking',
 'Drumming',
 'Dancing']

In [91]:
ActivityStats(original_df).num_days_in_df

365

In [92]:
ActivityStats(original_df).activities()

['Sleeping',
 'Grooming',
 'Toilet',
 'Eating',
 'Internet',
 'Cooking',
 'Cleaning',
 'Dishes',
 'Personal Time',
 'Games',
 'Driving',
 'Gym',
 'Shopping',
 'Family Time',
 'Walking',
 'Reading',
 'Programming',
 'Working',
 'Drinking',
 'Dating',
 'Errands and Appointments',
 'Studying',
 'Movies/TV',
 'Drumming',
 'Class',
 'Friends Time',
 'Phone',
 'Biking',
 'Dancing',
 'Concert/Event',
 'Swimming/Tanning',
 'Drawing',
 'Writing']

In [93]:
# # https://www.codegrepper.com/code-examples/python/how+to+create+dynamic+variable+names+in+python

# for activity in unique_activities:
#     globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')] = pd.DataFrame()
    
#     globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['Start'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'Start'])]
#     globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')]['End'] = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'End'])]
    
#     globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'Start'].mean())
#     globals()[f"median_{activity}_start".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'Start'].median())
#     globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'End'].mean())
#     globals()[f"median_{activity}_end".lower().replace('/', '_').replace(' ', '_')] = pd.to_datetime(df.loc[df['Activity'].str.contains(f"{activity}"), 'End'].median())
    
#     globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].sum()
#     globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).mean()
#     globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).median()
#     globals()[f"std_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = round(df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds']).std()
    
#     globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].min()
#     globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')] = df.loc[df['Activity'].str.contains(f"{activity}"), 'Duration in Seconds'].max()
    
#     globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] = len(globals()[f"average_{activity}_df".lower().replace('/', '_').replace(' ', '_')])

In [94]:
# for activity in unique_activities:
#     globals()[f"{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]))[-8:]
#     globals()[f"{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]).days)
#     globals()[f"total_{activity}".lower().replace('/', '_').replace(' ', '_')] = globals()[f"{activity}_daydelta".lower().replace('/', '_').replace(' ', '_')] + ':' + globals()[f"{activity}_timedelta".lower().replace('/', '_').replace(' ', '_')]
    
# #     globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
# #     globals()[f"mean_{activity}_durations_d".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]).days)
# #     globals()[f"mean_{activity}_durations_dhms".lower().replace('/', '_').replace(' ', '_')] = globals()[f"mean_{activity}_durations_d".lower().replace('/', '_').replace(' ', '_')] + ':' + globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')]

# #     globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]))[-8:]
# #     globals()[f"median_{activity}_durations_d".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]).days)
# #     globals()[f"median_{activity}_durations_dhms".lower().replace('/', '_').replace(' ', '_')] = globals()[f"median_{activity}_durations_d".lower().replace('/', '_').replace(' ', '_')] + ':' + globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')]
    
# #     globals()[f"mean_{activity}_start_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')]))[-8:]
# #     globals()[f"mean_{activity}_end_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=globals()[f"mean_{activity}_end".lower().replace('/', '_').replace(' ', '_')]))[-8:]

#     globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"mean_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]

#     globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"median_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
    
#     globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"min_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]
#     globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')] = str(pd.Timedelta(seconds=round(globals()[f"max_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])))[-8:]


In [95]:
# days_in_df = pd.DataFrame(df['Day'].unique(), columns=['Day'])
# num_days_in_df = len(days_in_df)
# num_days_in_df

In [96]:
# print("Standard deviations of each Activity in seconds\n")
# for activity in unique_activities:
#     print(f"{activity}: ", round(globals()[f"sstd_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]))

In [97]:
# print("Standard deviations of each Activity in D:H:M:S\n")
# for activity in unique_activities:
#     print(f"{activity}:", globals()[f"std_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')])

In [98]:
# print("Min duration of each Activity \n")
# for activity in unique_activities:
#     print(f"{activity}:", globals()[f"min_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')])

In [99]:
# print("Max duration of each Activity \n")
# for activity in unique_activities:
#     print(f"{activity}:", globals()[f"max_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')])

In [100]:
# print("Mean number of Instances of Activity per Day \n")
# for activity in unique_activities:
#     print(f"{activity}", round(globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')] / num_days_in_df, 2))

In [101]:
# print("Mean Time per Instance of Activity \n")
# for activity in unique_activities:
#     print(f"{activity}:", globals()[f"mean_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')])

In [102]:
print("Mean Start Times in Days:Hours:Minutes:Seconds per Instance of Activity \n")
for activity in unique_activities:
    print(f"{activity}:", globals()[f"mean_{activity}_start".lower().replace('/', '_').replace(' ', '_')])

Mean Start Times in Days:Hours:Minutes:Seconds per Instance of Activity 

Sleeping: 2021-07-05 19:09:27.250585600
Grooming: 2021-07-13 17:59:53.875342336
Toilet: 2021-07-14 11:33:50.345673984
Eating: 2021-07-13 01:50:53.625874176
Internet: 2021-07-05 07:48:34.877068544
Cooking: 2021-08-01 17:55:24.566197248
Cleaning: 2021-07-10 00:56:16.880952576
Dishes: 2021-08-06 21:10:14.461538304
Personal Time: 2021-07-03 22:45:26.432314368
Games: 2021-06-20 19:11:05.339805696
Driving: 2021-08-10 22:56:14.354092544
Gym: 2021-09-11 09:31:27.778523648
Shopping: 2021-07-11 08:02:11.488371968
Family Time: 2021-07-15 19:10:11.014286080
Walking: 2021-07-15 19:06:29.317327872
Reading: 2021-06-03 10:18:03.507042304
Programming: 2021-06-04 17:59:36.706790144
Working: 2021-06-24 19:59:24.231214080
Drinking: 2021-07-26 01:14:00.800000
Dating: 2021-06-02 03:37:18.250000128
Errands and Appointments: 2021-06-19 11:27:07.973684224
Studying: 2021-10-19 03:21:16.747126528
Movies/TV: 2021-08-06 17:24:52.959459328
Dr

In [103]:
# print("Median Time per Instance of Activity \n")
# for activity in unique_activities:
#     print(f"{activity}:", globals()[f"median_{activity}_durations_hms".lower().replace('/', '_').replace(' ', '_')])

In [104]:
# print("Total Number of Instances of Activity \n")
# for activity in unique_activities:
#     print(f"{activity}:", globals()[f"total_num_instance_{activity}".lower().replace('/', '_').replace(' ', '_')])

In [105]:
# total_seconds_list = []
# print("Total Seconds of each Activity \n")
# for activity in unique_activities:
#     total_seconds_list.append([f"{activity}", globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')]])
#     #print(f"{activity}:", globals()[f"sum_{activity}_durations".lower().replace('/', '_').replace(' ', '_')])
    
# total_seconds_list = sorted(total_seconds_list, key=lambda x: x[1], reverse=True)
# [i for i in total_seconds_list]

In [106]:
# unique_activities_sorted = sorted(total_seconds_list, key=lambda x: x[1], reverse=True)
# unique_activities_sorted = [i[0] for i in unique_activities_sorted]
# unique_activities_sorted

In [107]:
# print("Total Days:Hours:Minutes:Seconds of each Activity \n")
# for activity in unique_activities:
#     print(f"{activity}:", globals()[f"total_{activity}".lower().replace('/', '_').replace(' ', '_')])

# Start Times for hoverdata

In [108]:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.strftime.html
print(type(df['Start'][0]))

#df['Start Times'] = df['Start Times'].astype(str)
df['Start Times'] = df['Start'].dt.strftime('%H:%M:%S')
print(type(df['Start Times'][0]))
df

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>


,Group,Activity,Start,End,Day,Duration in Seconds,Duration,Start Times
0,Computer,Internet,1970-01-01 23:30:41,1970-01-01 23:59:59,2020-12-31,1758.0,00:29:18,23:30:41
9853,Computer,Internet,1970-01-01 00:00:01,1970-01-01 00:37:23,2021-01-01,2242.0,00:37:22,00:00:01
9852,NaN,Toilet,1970-01-01 00:37:35,1970-01-01 00:38:40,2021-01-01,65.0,00:01:05,00:37:35
9851,Computer,Internet,1970-01-01 00:42:17,1970-01-01 00:47:11,2021-01-01,294.0,00:04:54,00:42:17
9850,Computer,Games,1970-01-01 00:47:12,1970-01-01 08:10:35,2021-01-01,26603.0,07:23:23,00:47:12
...,...,...,...,...,...,...,...,...
421,NaN,Eating,1970-01-01 18:32:06,1970-01-01 18:33:35,2021-12-31,89.0,00:01:29,18:32:06
420,NaN,Toilet,1970-01-01 22:28:58,1970-01-01 22:30:00,2021-12-31,62.0,00:01:02,22:28:58
419,NaN,Grooming,1970-01-01 22:30:00,1970-01-01 22:32:21,2021-12-31,141.0,00:02:21,22:30:00
417,NaN,Sleeping,1970-01-01 22:33:38,1970-01-01 23:59:59,2021-12-31,5181.0,01:26:21,22:33:38


# Customdata

In [109]:
df.columns

Index(['Group', 'Activity', 'Start', 'End', 'Day', 'Duration in Seconds',
       'Duration', 'Start Times'],
      dtype='object')

In [110]:
customdata = ['Activity', 'Start Times', 'End', 'Duration']

In [111]:
# # Alternative to customdata with np.stack()
# # can add other customdata to the stack
# customdata = np.stack((df['Activity'],
#                        df['Start'], 
#                        df['End'],
#                        df['Duration'],),
#                        axis=1)
# print(customdata.shape)
# customdata[-3:]

# Zoom Levels

In [112]:
# https://plotly.com/python/time-series/#customizing-tick-label-formatting-by-zoom-level

In [113]:
# df['Width'] = 10000000
# df

# for i, d in enumerate(fig.data):
#     d.width = df[df['Activity']==d.name]['Width']


In [114]:
# fig.data[25].marker.line.width = None

In [115]:
# for i, bar in enumerate(fig.data):
#     print(i, fig.data[i].legendgroup, fig.data[i].marker.line)

# Activity Timeline Dashboard

In [116]:
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration,Start Times
0,Computer,Internet,1970-01-01 23:30:41,1970-01-01 23:59:59,2020-12-31,1758.0,00:29:18,23:30:41
9853,Computer,Internet,1970-01-01 00:00:01,1970-01-01 00:37:23,2021-01-01,2242.0,00:37:22,00:00:01
9852,NaN,Toilet,1970-01-01 00:37:35,1970-01-01 00:38:40,2021-01-01,65.0,00:01:05,00:37:35
9851,Computer,Internet,1970-01-01 00:42:17,1970-01-01 00:47:11,2021-01-01,294.0,00:04:54,00:42:17
9850,Computer,Games,1970-01-01 00:47:12,1970-01-01 08:10:35,2021-01-01,26603.0,07:23:23,00:47:12
...,...,...,...,...,...,...,...,...
421,NaN,Eating,1970-01-01 18:32:06,1970-01-01 18:33:35,2021-12-31,89.0,00:01:29,18:32:06
420,NaN,Toilet,1970-01-01 22:28:58,1970-01-01 22:30:00,2021-12-31,62.0,00:01:02,22:28:58
419,NaN,Grooming,1970-01-01 22:30:00,1970-01-01 22:32:21,2021-12-31,141.0,00:02:21,22:30:00
417,NaN,Sleeping,1970-01-01 22:33:38,1970-01-01 23:59:59,2021-12-31,5181.0,01:26:21,22:33:38


In [117]:
# for overlapping bars, i.e. bullet chart
# https://stackoverflow.com/questions/67129085/how-to-get-plotly-express-timeline-overlapped-bars-to-be-visible

# how to fix varying thickness of bars
# https://community.plotly.com/t/bars-in-timeline-chart-from-plotly-express-equivalent-of-gantt-differs-in-width/44221
# https://stackoverflow.com/questions/64665728/plotly-how-to-prevent-varying-thickness-of-bars-in-a-gantt-diagram
# https://stackoverflow.com/questions/67129085/how-to-get-plotly-express-timeline-overlapped-bars-to-be-visible
# https://github.com/plotly/plotly.py/issues/2947

# https://community.plotly.com/t/how-do-i-change-the-x-axis-representation-of-time-on-the-px-timeline/44834/4
# https://stackoverflow.com/questions/67319505/how-to-plot-multiple-timelines-in-one-graph
# https://stackoverflow.com/questions/66905297/plotly-is-it-possible-to-add-a-second-y-axis-to-plotly-timeline

# dropdown
# https://community.plotly.com/t/advanced-dropdown-menus-with-plotly/40922
# https://www.kaggle.com/benhamner/python-plotly-dropdown-demo
# https://www.kaggle.com/jrmistry/plotly-how-to-change-plot-data-using-dropdowns
# https://www.geeksforgeeks.org/how-to-make-dropdown-menus-in-plotly/
# https://stackoverflow.com/questions/59406167/plotly-how-to-filter-a-pandas-dataframe-using-a-dropdown-menu

title = 'Activity Timeline Dashboard'

fig = go.Figure()
fig = px.timeline(df, 
                  x_start="Start", 
                  x_end="End", 
                  y="Day", 
                  color='Activity', 
                  # https://stackoverflow.com/questions/59713016/plotly-express-how-to-fix-the-color-mapping-when-setting-color-by-column-name
                  color_discrete_map=colors,
                  category_orders={'Activity': [i for i in unique_activities_sorted]},
                  custom_data = customdata,
                  #template="plotly_dark",
                  width=1550,
                  height=900
                 )

# for i, bar in enumerate(fig.data):
#     fig.data[i].marker.line.width = .7

#fig.layout.barmode = 'group'    
    
fig.update_traces(hovertemplate =
                  #"Group: %{customdata[0]}<br>" +
                  "Activity: %{customdata[0]}<br>" +
                  "Day: %{y}<br>" +
                  "Start: %{customdata[1]}<br>" +
                  "End: %{x}<br>" +
                  "Duration: %{customdata[3]}" +
                  "<extra></extra>",
                  #width=10,
                 )

# https://plotly.com/python/marker-style/
#fig.add_trace(go.Scatter(x=df['Start'], y=df['Day'] , name='Starts', mode='markers', marker_color='#00CC96'))
#fig.add_trace(go.Scatter(x=df['End'], y=df['Day'] , name='Ends', mode='markers', marker_color='#EF553B'))

#fig.add_vline(x=median_sleep_start, line_dash="dash")
#fig.add_vline(x=median_sleep_end, line_dash="dash")

# https://github.com/d3/d3-time-format#locale_format change time format
fig.update_layout(xaxis=dict(title='Timestamp',
                             tickformat = '%H:%M:%S',
                             range=['1970-01-01 00:00:00','1970-01-01 23:59:59'],
                             fixedrange=True, # for keeping zoom level to entire days
                            ),
                  yaxis=dict(title='Day',
                             tickformat = '%B, %d, %Y',
                             dtick="M1",
                             #range=[]
                   ),
                  #hovermode='x unified',
                  #dragmode='pan',
                  title=dict(text=title, 
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                  legend=dict(itemclick='toggle',
                              itemdoubleclick='toggleothers',
                              
# # for putting legend below chart 
#                               orientation="h",
#                               yanchor="bottom",
#                               y=-.1,
#                               xanchor="right",
#                               x=1,
                  )
                 )

# https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
fig.update_layout(plot_bgcolor='#282F44',
                  paper_bgcolor ='#282F44',
                  font_color='#FFF',
                  #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                  #hovermode='x unified',
                 )

# fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
#         style={'height': '98vh',
#                'width': '98vw',
#               },
        
#         style={'height': '950px',
#                'width': '1600px',
#               },
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8050, debug=True)

Dash app running on http://127.0.0.1:8050/


# Data Table

In [118]:
# https://dash.plotly.com/datatable
from dash import dash_table

In [119]:
df = activity_df_transposed
df2 = activity_df_totals
title = "Data Table"
app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

app.layout = html.Div([
    html.H4('Activity Dataframe'),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} 
                 for i in df.columns],
        data=df.to_dict('records'),
        fixed_columns={'headers':True, 'data':1},
#        fixed_rows={'headers':True},
        style_cell={'textAlign':'left'},
        style_header={'backgroundColor': 'rgb(248, 248, 248)',
                      'fontWeight': 'bold',
                      'textAlign': 'right'},
        style_data={'font-family':'Open Sans, sans-serif',
                    'backgroundColor':"white",
                    'textAlign': 'right'},
        style_data_conditional=[{'if': {'row_index': 'odd'},
                                  'backgroundColor': 'rgb(248, 248, 248)'},
                                {'if': {'column_id': 'index'},
                                 'fontWeight': 'bold',
                                 'backgroundColor': 'rgb(248, 248, 248)'}
                                ],
        style_table={'overflowX': 'auto',
                     #'width': '900px',
                     'minWidth': '100%'},
        #page_size=10,
    ),
    html.Br(),
    
    html.H4('Total Activity Dataframe'),
    dash_table.DataTable(
        id='table2',
        columns=[{"name": i, "id": i} 
                 for i in df2.columns],
        data=df2.to_dict('records'),
#        fixed_columns={'headers':True, 'data':1},
#         fixed_rows={'headers':True, 'data':10},
        style_cell={'textAlign':'left'},
        style_header={'backgroundColor': 'rgb(248, 248, 248)',
                      'fontWeight': 'bold',
                      'textAlign': 'right'},
        style_data={'font-family':'Open Sans, sans-serif',
                    'backgroundColor':"white",
                    'textAlign': 'right'},
        style_data_conditional=[{'if': {'row_index': 'odd'},
                                  'backgroundColor': 'rgb(248, 248, 248)'},
                                {'if': {'column_id': 'index'},
                                 'fontWeight': 'bold',
                                 'backgroundColor': 'rgb(248, 248, 248)'}
                                ],
        style_table={'overflowX': 'auto',
                     'width': '600px',},
    )
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8060, debug=True)

Dash app running on http://127.0.0.1:8060/


/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/plotly/io/_json.py:467: UserWarning:

Discarding nonzero nanoseconds in conversion.



# Heatmap


In [117]:
df = original_df
df

,Group,Activity,Start,End,Day,Duration in Seconds,Duration
0,NaN,Sleeping,2022-01-01 00:00:00,2022-01-01 05:43:59,2021-12-31,25821.0,07:10:21
1,NaN,Grooming,2021-12-31 22:30:00,2021-12-31 22:32:21,2021-12-31,141.0,00:02:21
2,NaN,Toilet,2021-12-31 22:28:58,2021-12-31 22:30:00,2021-12-31,62.0,00:01:02
3,NaN,Eating,2021-12-31 18:32:06,2021-12-31 18:33:35,2021-12-31,89.0,00:01:29
4,Computer,Internet,2021-12-31 18:31:31,2021-12-31 22:28:54,2021-12-31,14243.0,03:57:23
...,...,...,...,...,...,...,...
9431,NaN,Toilet,2021-01-01 08:10:38,2021-01-01 08:12:25,2021-01-01,107.0,00:01:47
9432,Computer,Games,2021-01-01 00:47:12,2021-01-01 08:10:35,2021-01-01,26603.0,07:23:23
9433,Computer,Internet,2021-01-01 00:42:17,2021-01-01 00:47:11,2021-01-01,294.0,00:04:54
9434,NaN,Toilet,2021-01-01 00:37:35,2021-01-01 00:38:40,2021-01-01,65.0,00:01:05


In [118]:
# total number of activities per day
# with pd.option_context('display.max_rows', 400, 'display.max_columns', 5):
#      display(pd.DataFrame(df.groupby(df['Day']).agg('Duration').count().reset_index()))
total_num_activities_per_day = df.groupby(df['Day']).agg('Duration').count().reset_index()
total_num_activities_per_day = total_num_activities_per_day.rename({'Duration':'Counts'}, axis='columns')
total_num_activities_per_day

,Day,Counts
0,2020-12-31,1
1,2021-01-01,16
2,2021-01-02,21
3,2021-01-03,20
4,2021-01-04,19
...,...,...
361,2021-12-27,28
362,2021-12-28,30
363,2021-12-29,15
364,2021-12-30,25


In [119]:
# # number of instances of activity per day
# # with pd.option_context('display.max_rows', 400, 'display.max_columns', 5):
# #       display(pd.DataFrame(df.groupby(df.loc[df['Activity'].str.contains("Programming"), 'Day']).agg('Duration').count().reset_index()))
# programming_counts = df.groupby(df.loc[df['Activity'].str.contains("Programming"), 'Day']).agg('Duration').count().reset_index()
# programming_counts

In [120]:
# programming_sums = df.groupby(df.loc[df['Activity'].str.contains("Programming"), 'Day']).agg('Duration in Seconds').sum().reset_index()
# programming_sums

In [121]:
# programming_hms = str(pd.Timedelta(seconds=programming_sums['Duration in Seconds']))[-8:]
# programming_hms

In [122]:
# #df.groupby(df.loc[df['Activity'].str.contains("Programming"), 'Day']).agg('Duration').count().reset_index().merge(df.groupby(df.loc[df['Activity'].str.contains("Programming"), 'Day']).agg('Duration in Seconds').sum().reset_index())
# new_df = programming_counts.merge(programming_sums)
# new_df = new_df.reset_index()
# new_df

In [123]:
# df.groupby(df.loc[df['Activity'].str.contains("Programming"), 'Day']).agg({'Duration in Seconds':['count', 'sum']}).reset_index()

In [124]:
# new_df = pd.DataFrame()
# new_df['Day'] = df.loc[df['Activity'].str.contains("Programming"), 'Day']
# new_df['duration_seconds'] = df.loc[df['Activity'].str.contains("Programming"), 'Duration in Seconds']
# new_df['duration_in_hms'] = df.loc[df['Activity'].str.contains("Programming"), 'Duration']
# #new_df['instances_per_day'] = df.groupby(df.loc[df['Activity'].str.contains("Programming"), 'Day']).agg('Duration').count().reset_index()
# new_df

In [125]:
programming_seconds = [x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains("Programming"), 'Duration in Seconds'])]
programming_days = pd.to_datetime([x.value for x in pd.to_datetime(df.loc[df['Activity'].str.contains("Programming"), 'Day'])])
programming_days = [str(i)[:10] for i in programming_days]
#programming = [i for i in programming_days]
programming_df = pd.DataFrame()

programming_df['Days'] =  [pd.to_datetime(x) for x in programming_days]
programming_df['Seconds'] = [int(x) for x in programming_seconds]
#programming_df['Seconds'] = [pd.to_timedelta(x, unit='s') for x in programming_seconds]

programming_df

,Days,Seconds
0,2021-12-27,8643
1,2021-12-27,1783
2,2021-12-26,2401
3,2021-12-25,5724
4,2021-12-25,4822
...,...,...
319,2021-01-03,1375
320,2021-01-03,9978
321,2021-01-02,5399
322,2021-01-02,2705


In [126]:
# with pd.option_context('display.max_rows', 225, 'display.max_columns', 5):
#     display(programming_df)

In [127]:
# https://stackoverflow.com/questions/46444444/how-to-calculate-sum-of-column-per-day-in-pandas
grouped_programming_df = pd.DataFrame(programming_df.groupby(programming_df['Days']).agg('Seconds').sum()).reset_index()
grouped_programming_df
# have to interpolate the dates so there is entire year, 
# i.e, fill in blank days where you didn't have any programming seconds to 0, df should be size 365-366 days or so

,Days,Seconds
0,2021-01-01,8589
1,2021-01-02,8104
2,2021-01-03,17481
3,2021-01-04,17406
4,2021-01-07,11648
...,...,...
130,2021-12-20,7501
131,2021-12-24,11082
132,2021-12-25,16063
133,2021-12-26,2401


In [128]:
# for the counts of the instances of the activity per day
count_grouped_programming_df = pd.DataFrame(programming_df.groupby(programming_df['Days']).agg('Seconds').count()).reset_index()
count_grouped_programming_df = count_grouped_programming_df.rename({'Seconds':'Counts'}, axis='columns')
grouped_programming_df['Counts'] = count_grouped_programming_df['Counts']
grouped_programming_df

,Days,Seconds,Counts
0,2021-01-01,8589,1
1,2021-01-02,8104,2
2,2021-01-03,17481,4
3,2021-01-04,17406,5
4,2021-01-07,11648,3
...,...,...,...
130,2021-12-20,7501,1
131,2021-12-24,11082,4
132,2021-12-25,16063,4
133,2021-12-26,2401,1


In [129]:
print(type(grouped_programming_df['Days'][0]))
print(type(grouped_programming_df['Seconds'][0]))
print(type(grouped_programming_df['Counts'][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'numpy.int64'>
<class 'numpy.int64'>


In [130]:
end = pd.to_datetime(original_df['Day'][0])
print("End:", end)
print(type(end), '\n')
start = pd.to_datetime(original_df['Day'].iloc[-1])
print("Start:", start)
print(type(start))
# pd.date_range(start, end)

End: 2021-12-31 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'> 

Start: 2020-12-31 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [131]:
# https://www.kite.com/python/answers/how-to-insert-a-row-into-a-pandas-dataframe
# https://stackoverflow.com/questions/19324453/add-missing-dates-to-pandas-dataframe
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.asfreq.html#pandas.DataFrame.asfreq
row_df = pd.DataFrame({'Days':[start],
                      'Seconds':0})
grouped_programming_df = pd.concat([row_df, grouped_programming_df], ignore_index=True)
grouped_programming_df = grouped_programming_df.set_index('Days')
grouped_programming_df = grouped_programming_df.asfreq('D', fill_value=0).reset_index()
grouped_programming_df = grouped_programming_df.drop([0, 1]).reset_index()
grouped_programming_df['Seconds_hms'] = 0
for i in grouped_programming_df['Seconds'].index:
    grouped_programming_df['Seconds_hms'][i] = str(pd.Timedelta(seconds=grouped_programming_df['Seconds'][i]))[-8:]


# there is a more efficient way with .loc but idk how
#grouped_programming_df.loc[grouped_programming_df['Seconds'], 'Seconds_hms'] = str(pd.Timedelta(seconds=grouped_programming_df['Seconds']))[-8:]    
    
    
del grouped_programming_df['index']
grouped_programming_df

/tmp/ipykernel_228242/2626003984.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Days,Seconds,Counts,Seconds_hms
0,2021-01-02,8104,2.0,02:15:04
1,2021-01-03,17481,4.0,04:51:21
2,2021-01-04,17406,5.0,04:50:06
3,2021-01-05,0,0.0,00:00:00
4,2021-01-06,0,0.0,00:00:00
...,...,...,...,...
355,2021-12-23,0,0.0,00:00:00
356,2021-12-24,11082,4.0,03:04:42
357,2021-12-25,16063,4.0,04:27:43
358,2021-12-26,2401,1.0,00:40:01


In [132]:
print(type(grouped_programming_df['Days'][0]))
print(type(grouped_programming_df['Seconds'][0]))
print(type(grouped_programming_df['Counts'][0]))
print(type(grouped_programming_df['Seconds_hms'][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'str'>


In [133]:
# with pd.option_context('display.max_rows', 500, 'display.max_columns', 5):
#      display(grouped_programming_df)

In [134]:
# # for *slice* style heatmap, maybe you could stack them into one activity per y for all activities in df

# df = programming_df
# title = f"Programming Heatmap"
# #bins= 

# app = jupyter_dash.JupyterDash(__name__, 
#                                external_stylesheets=external_stylesheets, 
#                                title=f"{title} Dashboard")

# fig = go.Figure(data=go.Heatmap(x=[pd.to_datetime(i) for i in grouped_programming_df['Days']],
#                                 z=[grouped_programming_df['Seconds']],
#                                 y=['Matthew'],
#                                 colorscale='Viridis',
#                                ))

# fig.update_traces(#ybins=dict(size=bins),
#                   xbins=dict(size=86400000.0),
#                   selector=dict(type='histogram2d'))

# app.layout = html.Div([
#         dcc.Graph(
#             figure=fig
#         )
#     ])

# if __name__ == '__main__':
#     app.run_server(mode='external', debug=True, port=9050)

In [135]:
# https://community.plotly.com/t/colored-calendar-heatmap-in-dash/10907/17
# real cool, must cite him https://community.plotly.com/u/bendichter
# https://plotly.com/python/heatmaps/#heatmap-with-plotlyexpress-and-pximshow

#***** trying to pretty up the hovertext****#
# try and make it so it automatically detects if the year is a leapyear

import datetime
import plotly.graph_objs as go
import numpy as np
import dash_core_components as dcc
import dash_html_components as html
import dash

title = f"Programming Heatmap"

app = jupyter_dash.JupyterDash(__name__, 
                               external_stylesheets=external_stylesheets, 
                               title=f"{title} Dashboard")

def display_year(z,
                 year: int = None,
                 month_lines: bool = True,
                 fig=None,
                 row: int = None):
    
    #replace with more generic version, for all activities
    customdata = grouped_programming_df
    
    if year is None:
        year = datetime.datetime.now().year
    
    #replace 366 with 365 for regular years
    data = np.ones(366) * np.nan
    data[:len(z)] = z
    

    d1 = datetime.date(year, 1, 1)
    d2 = datetime.date(year, 12, 31)

    delta = d2 - d1
    
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    # for leap years you have to make feb 29 else 28
    month_days =   [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_positions = (np.cumsum(month_days) - 15)/7
    
    #gives me a list with datetimes for each day a year
    dates_in_year = [d1 + datetime.timedelta(i) for i in range(delta.days+1)] 
    #gives [0,1,2,3,4,5,6,0,1,2,3,4,5,6,…] (ticktext in xaxis dict translates this to weekdays
    weekdays_in_year = [i.weekday() for i in dates_in_year] 
    
    #gives [1,1,1,1,1,1,1,2,2,2,2,2,2,2,…] name is self-explanatory
    weeknumber_of_dates = [int(i.strftime("%V")) if not (int(i.strftime("%V")) == 1 and i.month == 12) else 53
                           for i in dates_in_year] 
    #gives something like list of strings like ‘2018-01-25’ for each date. Used in data trace to make good hovertext.
    text = [str(i) for i in dates_in_year]
    #4cc417 green #347c17 dark green
    colorscale=[[False, '#eeeeee'], [True, '#76cf63']]
    
    # handle end of year
    
    data = [
        go.Heatmap(
            x=weeknumber_of_dates,
            y=weekdays_in_year,
            z=data,
            text=text,
            hoverinfo=['text', 'z'],
            xgap=2, # this
            ygap=2, # and this is for width between squares
            showscale=False,
            colorscale=colorscale,
            customdata=customdata, # replace with more generic version for all activities
            hovertemplate =
            "Day: %{text}<br>" +
            "Duration: %{customdata[3]}<br>" +
            "Counts: %{customdata[2]}<br>" +
            "<extra></extra>",
        )]
        
    if month_lines:
        kwargs = dict(
            mode='lines',
            line=dict(
                color='#9e9e9e',
                width=2, # num pixels for the line thickness
            ),
            hoverinfo='skip')
        for date, dow, wkn in zip(dates_in_year,
                                  weekdays_in_year,
                                  weeknumber_of_dates):
            if date.day == 1:
                data += [go.Scatter(
                    x=[wkn-.5, wkn-.5],
                    y=[dow-.5, 6.5],
                    **kwargs)]
                if dow:
                    data += [
                    go.Scatter(
                        x=[wkn-.5, wkn+.5],
                        y=[dow-.5, dow - .5],
                        **kwargs),
                    go.Scatter(
                        x=[wkn+.5, wkn+.5],
                        y=[dow-.5, -.5],
                        **kwargs)]
                        
    layout = go.Layout(
        title=title,
        height=250,
        yaxis=dict(
            showline=False, showgrid=False, zeroline=False,
            tickmode='array',
            ticktext=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            tickvals=[0, 1, 2, 3, 4, 5, 6],
            autorange="reversed"),
        xaxis=dict(
            showline=False, showgrid=False, zeroline=False,
            tickmode='array',
            ticktext=month_names,
            tickvals=month_positions),
        font={#'size':10, 
              #'color':'#9e9e9e'
        },
        plot_bgcolor=('#fff'),
        margin = dict(t=40),
        showlegend=False)

    if fig is None:
        fig = go.Figure(data=data, layout=layout)
    else:
        fig.add_traces(data, rows=[(row+1)]*len(data), cols=[1]*len(data))
        fig.update_layout(layout)
        fig.update_xaxes(layout['xaxis'])
        fig.update_yaxes(layout['yaxis'])
    return fig

def display_years(z, years):
    fig = make_subplots(rows=len(years), cols=1, subplot_titles=years)
    for i, year in enumerate(years):
        # replace 366 with 365 for regular years
        data = z[i*366 : (i+1)*366]
        display_year(data, year=year, fig=fig, row=i)
        fig.update_layout(height=250*len(years))
    return fig

#z = grouped_internet_df['internet_durations_seconds']
z = grouped_programming_df['Seconds']
# need years to be a tuple, so if only 1 year you have to have a trailing comma: ','
# need to configure customdata as argument, get z from that maybe
# need to configure colorscale as argument
# configure autodetection of leap year, change feb to 29 if so, and the data to 366
fig = display_years(z, (2020,))

app.layout = html.Div([
        dcc.Graph(
            figure=fig
        )
    ])

if __name__ == '__main__':
    app.run_server(mode='external', debug=True, port=8070)

/tmp/ipykernel_228242/98122196.py:11: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/tmp/ipykernel_228242/98122196.py:12: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



Dash app running on http://127.0.0.1:8070/


In [ ]:
grouped_internet_df

# Activity Timeline Barcharts

In [156]:
df = activity_correlations_df
activity_correlations_df

,Day,sleeping_durations_seconds,sleeping_counts,sleeping_durations,sleeping_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations,working_durations_seconds_7_day_rolling_mean,internet_durations_seconds,...,biking_durations,biking_durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations,drumming_durations_seconds_7_day_rolling_mean,dancing_durations_seconds,dancing_counts,dancing_durations,dancing_durations_seconds_7_day_rolling_mean
0,2021-01-01,21093.0,1.0,05:51:33,0.000000,44301.0,1.0,12:18:21,0.0,14943,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
1,2021-01-02,26339.0,1.0,07:18:59,0.000000,43800.0,1.0,12:10:00,0.0,9385,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
2,2021-01-03,28437.0,1.0,07:53:57,0.000000,43899.0,1.0,12:11:39,0.0,7748,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
3,2021-01-04,42197.0,1.0,11:43:17,0.000000,0.0,0.0,00:00:00,0.0,17229,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
4,2021-01-05,18942.0,1.0,05:15:42,0.000000,0.0,0.0,00:00:00,0.0,9779,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2021-12-28,33779.0,2.0,09:22:59,28383.285714,0.0,0.0,0,0.0,19175,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
362,2021-12-29,53196.0,2.0,14:46:36,30296.714286,0.0,0.0,0,0.0,15681,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
363,2021-12-30,36785.0,2.0,10:13:05,31983.142857,0.0,0.0,0,0.0,46155,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0
364,2021-12-31,25821.0,1.0,07:10:21,32404.142857,0.0,0.0,0,0.0,39865,...,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0


In [157]:
df_counts = [i for i in df.columns if i.endswith('counts')]
df_counts

['sleeping_counts',
 'working_counts',
 'internet_counts',
 'programming_counts',
 'games_counts',
 'family_time_counts',
 'eating_counts',
 'grooming_counts',
 'friends_time_counts',
 'gym_counts',
 'reading_counts',
 'movies_tv_counts',
 'toilet_counts',
 'studying_counts',
 'driving_counts',
 'walking_counts',
 'class_counts',
 'dating_counts',
 'cooking_counts',
 'dishes_counts',
 'personal_time_counts',
 'cleaning_counts',
 'shopping_counts',
 'writing_counts',
 'errands_and_appointments_counts',
 'drawing_counts',
 'concert_event_counts',
 'phone_counts',
 'drinking_counts',
 'swimming_tanning_counts',
 'biking_counts',
 'drumming_counts',
 'dancing_counts']

In [158]:
df_durations_seconds = [i for i in df.columns if i.endswith('seconds')]
df_durations_seconds

['sleeping_durations_seconds',
 'working_durations_seconds',
 'internet_durations_seconds',
 'programming_durations_seconds',
 'games_durations_seconds',
 'family_time_durations_seconds',
 'eating_durations_seconds',
 'grooming_durations_seconds',
 'friends_time_durations_seconds',
 'gym_durations_seconds',
 'reading_durations_seconds',
 'movies_tv_durations_seconds',
 'toilet_durations_seconds',
 'studying_durations_seconds',
 'driving_durations_seconds',
 'walking_durations_seconds',
 'class_durations_seconds',
 'dating_durations_seconds',
 'cooking_durations_seconds',
 'dishes_durations_seconds',
 'personal_time_durations_seconds',
 'cleaning_durations_seconds',
 'shopping_durations_seconds',
 'writing_durations_seconds',
 'errands_and_appointments_durations_seconds',
 'drawing_durations_seconds',
 'concert_event_durations_seconds',
 'phone_durations_seconds',
 'drinking_durations_seconds',
 'swimming_tanning_durations_seconds',
 'biking_durations_seconds',
 'drumming_durations_seco

In [159]:
df_durations = [i for i in df.columns if i.endswith('durations')]
df_durations

['sleeping_durations',
 'working_durations',
 'internet_durations',
 'programming_durations',
 'games_durations',
 'family_time_durations',
 'eating_durations',
 'grooming_durations',
 'friends_time_durations',
 'gym_durations',
 'reading_durations',
 'movies_tv_durations',
 'toilet_durations',
 'studying_durations',
 'driving_durations',
 'walking_durations',
 'class_durations',
 'dating_durations',
 'cooking_durations',
 'dishes_durations',
 'personal_time_durations',
 'cleaning_durations',
 'shopping_durations',
 'writing_durations',
 'errands_and_appointments_durations',
 'drawing_durations',
 'concert_event_durations',
 'phone_durations',
 'drinking_durations',
 'swimming_tanning_durations',
 'biking_durations',
 'drumming_durations',
 'dancing_durations']

In [160]:
df_durations_seconds_7_day_rolling_mean = [i for i in df.columns if i.endswith('mean')]
df_durations_seconds_7_day_rolling_mean

['sleeping_durations_seconds_7_day_rolling_mean',
 'working_durations_seconds_7_day_rolling_mean',
 'internet_durations_seconds_7_day_rolling_mean',
 'programming_durations_seconds_7_day_rolling_mean',
 'games_durations_seconds_7_day_rolling_mean',
 'family_time_durations_seconds_7_day_rolling_mean',
 'eating_durations_seconds_7_day_rolling_mean',
 'grooming_durations_seconds_7_day_rolling_mean',
 'friends_time_durations_seconds_7_day_rolling_mean',
 'gym_durations_seconds_7_day_rolling_mean',
 'reading_durations_seconds_7_day_rolling_mean',
 'movies_tv_durations_seconds_7_day_rolling_mean',
 'toilet_durations_seconds_7_day_rolling_mean',
 'studying_durations_seconds_7_day_rolling_mean',
 'driving_durations_seconds_7_day_rolling_mean',
 'walking_durations_seconds_7_day_rolling_mean',
 'class_durations_seconds_7_day_rolling_mean',
 'dating_durations_seconds_7_day_rolling_mean',
 'cooking_durations_seconds_7_day_rolling_mean',
 'dishes_durations_seconds_7_day_rolling_mean',
 'personal_ti

In [161]:
# sleep_rolling =  round(df['sleeping_durations_seconds'].rolling(rolling).mean())
# sleep_rolling

In [162]:
# how to make a new color dictionary based on the original dictionary, 
# could make another one for other df's.columns e.g. <current> line 5 of this cell
correlation_colors = {}
for k, v in colors.items():
#     print(k)
    for col in activity_correlations_df.columns:
        if k.lower().replace('/', '_').replace(' ', '_') in col:
#             print(col)
            correlation_colors[col] = v
correlation_colors

{'working_durations_seconds': '#D50000',
 'working_counts': '#D50000',
 'working_durations': '#D50000',
 'working_durations_seconds_7_day_rolling_mean': '#D50000',
 'sleeping_durations_seconds': '#2196F3',
 'sleeping_counts': '#2196F3',
 'sleeping_durations': '#2196F3',
 'sleeping_durations_seconds_7_day_rolling_mean': '#2196F3',
 'grooming_durations_seconds': '#80D8FF',
 'grooming_counts': '#80D8FF',
 'grooming_durations': '#80D8FF',
 'grooming_durations_seconds_7_day_rolling_mean': '#80D8FF',
 'internet_durations_seconds': '#00C853',
 'internet_counts': '#00C853',
 'internet_durations': '#00C853',
 'internet_durations_seconds_7_day_rolling_mean': '#00C853',
 'movies_tv_durations_seconds': '#00C853',
 'movies_tv_counts': '#00C853',
 'movies_tv_durations': '#00C853',
 'movies_tv_durations_seconds_7_day_rolling_mean': '#00C853',
 'programming_durations_seconds': '#00E676',
 'programming_counts': '#00E676',
 'programming_durations': '#00E676',
 'programming_durations_seconds_7_day_rollin

In [163]:
# customdata = df_durations
# customdata

In [164]:
# [[i for i in df[col]] for col in df[df_durations].columns]

In [165]:
title = 'Activity Timeline Stacked Barchart'

df = activity_correlations_df
customdata=[df[col] for col in df[df_durations].columns],

rolling = 7

fig = px.bar(df, 
             x=df['Day'], 
             y=[df[col] for col in df[df_durations_seconds].columns],
             color_discrete_map=correlation_colors,
             title=title,
            )

fig.update_traces(customdata=customdata,
                  hovertemplate =
#                   "Activity: {}<br>" +
                  "Day: %{x}<br>" +
                  "Duration in Seconds: %{y}<br>" +
#                   "Duration: %{customdata}<br>" +
                  "<extra></extra>",
                  #width=10,
                 )

#fig = go.Figure()
# for col in df[df_durations].columns:
#     fig.add_trace(go.Bar(x=df["Day"], 
#                          y=df[col],
#                          name=df[col].name,
# #                         marker_color=df[col], 
#                          marker=dict(color_discrete_map=correlation_colors,)
#                         ))

# https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
fig.update_layout(barmode='stack',
                  plot_bgcolor='#282F44',
                  paper_bgcolor ='#282F44',
                  font_color='#FFF',
                  #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                  #hovermode='x unified',
                  
                  # https://github.com/d3/d3-time-format#locale_format change time format
                  xaxis=dict(title='Day',
                            ),
                  yaxis=dict(title='Activities',
                             fixedrange=True, # for keeping zoom level to entire days
                            ),
                  #hovermode='x unified',
                  #dragmode='pan',
                  title=dict(text=title,
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                  legend=dict(title='Activities',
                              itemclick='toggle',
                              itemdoubleclick='toggleothers',
                              # for putting legend below chart 
#                               orientation="h",
#                               yanchor="bottom",
#                               y=-.1,
#                               xanchor="right",
#                               x=1,
                             ),
                 )

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")
              
app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               'width': '98vw',
              },
        
#         style={'height': '950px',
#                'width': '1600px',
#               },
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8080, debug=True)

Dash app running on http://127.0.0.1:8080/


# Activity Timeline Scatter

In [166]:
title = 'Activity Timeline Scatter Plot'

df = activity_correlations_df
customdata=[df[col] for col in df[df_durations].columns],

rolling = 7

fig = px.scatter(df, 
                 x=df['Day'], 
                 y=[df[col] for col in df[df_durations_seconds].columns],
                 color_discrete_map=correlation_colors,
                 title=title,
                 # https://plotly.com/python/linear-fits/
                 trendline="ols",
#                  trendline="rolling",
#                  trendline_options=dict(window=rolling),
                )

fig.update_traces(customdata=customdata,
                  hovertemplate =
#                   "Activity: {}<br>" +
                  "Day: %{x}<br>" +
                  "Duration in Seconds: %{y}<br>" +
#                   "Duration: %{customdata}<br>" +
                  "<extra></extra>",
                  #width=10,
                 )

#fig = go.Figure()
# for col in df[df_durations].columns:
#     fig.add_trace(go.Bar(x=df["Day"], 
#                          y=df[col],
#                          name=df[col].name,
# #                         marker_color=df[col], 
#                          marker=dict(color_discrete_map=correlation_colors,)
#                         ))

# https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
fig.update_layout(barmode='stack',
                  plot_bgcolor='#282F44',
                  paper_bgcolor ='#282F44',
                  font_color='#FFF',
                  #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                  #hovermode='x unified',
                  
                  # https://github.com/d3/d3-time-format#locale_format change time format
                  xaxis=dict(title='Day',
                            ),
                  yaxis=dict(title='Activities',
#                              fixedrange=True, # for keeping zoom level to entire days
                            ),
                  #hovermode='x unified',
                  #dragmode='pan',
                  title=dict(text=title, 
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                  legend=dict(title='Activities',
                              itemclick='toggle',
                              itemdoubleclick='toggleothers',
                              # for putting legend below chart 
#                               orientation="h",
#                               yanchor="bottom",
#                               y=-.1,
#                               xanchor="right",
#                               x=1,
                             ),
                 )

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")
              
app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               'width': '98vw',
              },
        
#         style={'height': '950px',
#                'width': '1600px',
#               },
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8095, debug=True)

Dash app running on http://127.0.0.1:8095/


In [167]:
# title = 'Activity Timeline Scatter Plot'

# df = activity_correlations_df
# rolling = 7

# fig = go.Figure()

# for col in df[df_durations_seconds].columns:
#     y_rolling = round(df[col].rolling(rolling).mean())
# #     fig.add_trace(go.Scatter(x=df["Day"], 
# #                              y=df[col],
# #                              name=df[col].name,
# #                              #mode='markers',
# #                             )),
#     fig.add_trace(go.Scatter(x=df["Day"], 
#                              y=y_rolling,
#                              name=f"{rolling} Day Rolling Average {df[col].name}",
#                             ))

# # for i in df.columns[1:3]:
# #     if i.endswith('seconds'):
# #         fig.add_trace(go.Scatter(x=df["Day"], 
# #                                  y=df[i],
# #                                  mode='markers',
# #                                 ))

# app = jupyter_dash.JupyterDash(__name__,
#                                external_stylesheets=external_stylesheets,
#                                title=f"{title}")
              
# app.layout = html.Div([
#     dcc.Graph(
#         figure=fig,
        
#         #https://plotly.com/python/setting-graph-size/
#         #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
# #         style={'height': '98vh',
# #                'width': '98vw',
# #               },
        
# #         style={'height': '950px',
# #                'width': '1600px',
# #               },
#     ),
# ])

# if __name__ == '__main__':
#     app.run_server(mode='external', port=8060, debug=True)

# Correlation Matrix

In [136]:
# correlation_matrix

In [137]:
rho

,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,programming_durations_seconds,...,swimming_tanning_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,dancing_durations_seconds,dancing_counts,dancing_durations_seconds_7_day_rolling_mean
sleeping_durations_seconds,1.000,0.327,0.256,-0.353,-0.309,-0.040,-0.004,-0.061,-0.041,-0.226,...,-0.012,0.045,0.045,NaN,-0.015,-0.014,-0.028,0.076,0.076,NaN
sleeping_counts,0.327,1.000,0.140,-0.440,-0.399,-0.098,0.297,0.269,0.016,-0.306,...,-0.047,0.049,0.049,NaN,-0.009,-0.010,0.011,-0.034,-0.034,NaN
sleeping_durations_seconds_7_day_rolling_mean,0.256,0.140,1.000,-0.052,-0.024,0.230,0.030,0.006,0.237,-0.165,...,-0.013,0.012,0.012,NaN,0.002,0.003,0.023,0.026,0.026,NaN
working_durations_seconds,-0.353,-0.440,-0.052,1.000,0.951,0.289,-0.515,-0.506,-0.047,0.612,...,-0.018,-0.058,-0.058,NaN,-0.101,-0.101,-0.009,-0.058,-0.058,NaN
working_counts,-0.309,-0.399,-0.024,0.951,1.000,0.238,-0.494,-0.479,-0.029,0.567,...,-0.035,-0.061,-0.061,NaN,-0.105,-0.105,-0.001,-0.061,-0.061,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
drumming_counts,-0.014,-0.010,0.003,-0.101,-0.105,-0.016,0.044,0.068,0.114,-0.057,...,-0.007,-0.005,-0.005,NaN,0.997,1.000,0.355,-0.005,-0.005,NaN
drumming_durations_seconds_7_day_rolling_mean,-0.028,0.011,0.023,-0.009,-0.001,-0.028,0.061,0.051,0.154,-0.005,...,-0.012,-0.008,-0.008,NaN,0.343,0.355,1.000,-0.008,-0.008,NaN
dancing_durations_seconds,0.076,-0.034,0.026,-0.058,-0.061,-0.146,-0.026,-0.019,-0.065,-0.033,...,-0.004,-0.003,-0.003,NaN,-0.005,-0.005,-0.008,1.000,1.000,NaN
dancing_counts,0.076,-0.034,0.026,-0.058,-0.061,-0.146,-0.026,-0.019,-0.065,-0.033,...,-0.004,-0.003,-0.003,NaN,-0.005,-0.005,-0.008,1.000,1.000,NaN


In [138]:
# # how to make subset of index and columns
# rho.loc[df_durations_seconds_7_day_rolling_mean][df_durations_seconds_7_day_rolling_mean]

# # or by adding together several other subsets
# rho.loc[df_durations_seconds+df_counts][df_durations_seconds+df_counts]

In [139]:
# https://community.plotly.com/t/correlation-heatmap-in-plotly/50884
# https://community.plotly.com/t/correlation-plot-with-mask/29185/3

title = 'Correlation Heatmap'

# can make subset by rho[<subset>] e.g. rho.loc[df_durations_seconds][df[durations_seconds]
#df = rho.loc[df_durations_seconds_7_day_rolling_mean][df_durations_seconds_7_day_rolling_mean]
#df = rho.loc[df_durations_seconds+df_counts][df_durations_seconds+df_counts]
df = rho.loc[df_durations_seconds][df_durations_seconds]

#df = rho
mask = np.triu(np.ones_like(df, dtype=bool))
df = df.mask(mask)

fig = go.Figure()
fig = fig.add_trace(go.Heatmap(z=df.values,
                               x=df.index,
                               y=df.columns,
                               #square=True,
                               colorscale=px.colors.diverging.RdBu,
                               reversescale=True,
                               zmin=-1,
                               zmax=1,
                               xgap = 1,
                               ygap = 1,
                              ))

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")

fig.update_xaxes(tickangle=-45,
                 #mirror='allticks', 
                 #side='top',
                )
fig.update_layout(title=title,
                  yaxis_autorange='reversed',
                  xaxis_showgrid=False,
                  yaxis_showgrid=False,
                 )
              
app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               'width': '98vw',
              },
        
#         style={'height': '1100px',
#                'width': '1600px',
#               },
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8100, debug=True)

Dash app running on http://127.0.0.1:8100/


In [168]:
df_durations

['sleeping_durations',
 'working_durations',
 'internet_durations',
 'programming_durations',
 'games_durations',
 'family_time_durations',
 'eating_durations',
 'grooming_durations',
 'friends_time_durations',
 'gym_durations',
 'reading_durations',
 'movies_tv_durations',
 'toilet_durations',
 'studying_durations',
 'driving_durations',
 'walking_durations',
 'class_durations',
 'dating_durations',
 'cooking_durations',
 'dishes_durations',
 'personal_time_durations',
 'cleaning_durations',
 'shopping_durations',
 'writing_durations',
 'errands_and_appointments_durations',
 'drawing_durations',
 'concert_event_durations',
 'phone_durations',
 'drinking_durations',
 'swimming_tanning_durations',
 'biking_durations',
 'drumming_durations',
 'dancing_durations']

# Activity Timeline Histogram

In [169]:
title = 'Activity Timeline Histogram'

df = activity_correlations_df
customdata=[df[col] for col in df[df_durations].columns],

rolling = 7

fig = px.histogram(df, 
                   #x=df['Day'], 
                   x=[df[col] for col in df[df_durations_seconds].columns],
                   color_discrete_map=correlation_colors,
                   title=title,
                  )

fig.update_traces(customdata=customdata,
                  hovertemplate =
#                   "Activity: {}<br>" +
                  "Counts: %{y}<br>" +
                  "Duration in Seconds: %{x}<br>" +
#                   "Duration: %{customdata}<br>" +
                  "<extra></extra>",
                  #width=10,
                 )

#fig = go.Figure()
# for col in df[df_durations].columns:
#     fig.add_trace(go.Bar(x=df["Day"], 
#                          y=df[col],
#                          name=df[col].name,
# #                         marker_color=df[col], 
#                          marker=dict(color_discrete_map=correlation_colors,)
#                         ))

# https://stackoverflow.com/questions/61406968/change-colors-in-python-dash-plotly-theme
fig.update_layout(barmode='stack',
                  plot_bgcolor='#282F44',
                  paper_bgcolor ='#282F44',
                  font_color='#FFF',
                  #modebar_add="v1hovermode",#+hoverclosest+hovercompare+togglehover+togglespikelines",
                  #hovermode='x unified',
                  
                  # https://github.com/d3/d3-time-format#locale_format change time format
                  xaxis=dict(title='Day',
                            ),
                  yaxis=dict(title='Activities',
#                              fixedrange=True, # for keeping zoom level to entire days
                            ),
                  #hovermode='x unified',
                  #dragmode='pan',
                  title=dict(text=title, 
                             yanchor="top",
                             y=.98,
                             xanchor="left",
                             x=.05),
                  legend=dict(title='Activities',
                              itemclick='toggle',
                              itemdoubleclick='toggleothers',
                              # for putting legend below chart 
#                               orientation="h",
#                               yanchor="bottom",
#                               y=-.1,
#                               xanchor="right",
#                               x=1,
                             ),
                 )

app = jupyter_dash.JupyterDash(__name__,
                               external_stylesheets=external_stylesheets,
                               title=f"{title}")
              
app.layout = html.Div([
    dcc.Graph(
        figure=fig,
        
        #https://plotly.com/python/setting-graph-size/
        #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
        style={'height': '98vh',
               'width': '98vw',
              },
        
#         style={'height': '950px',
#                'width': '1600px',
#               },
    ),
])

if __name__ == '__main__':
    app.run_server(mode='external', port=8110, debug=True)

Dash app running on http://127.0.0.1:8110/


### Old stuff that you probably don't need

In [ ]:
# df['Day'] = pd.to_datetime(df.Day)

# # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
# df["Day"] = pd.to_datetime(df["Day"]).dt.dayofyear

# day = df['Day']
# day

In [ ]:
# df['From'] = pd.to_datetime(df.From)
# df["From"] = df["From"].apply(lambda x: x.replace(year=2020, month=1, day=1))

# start = df['From']
# start

In [ ]:
# df['To'] = pd.to_datetime(df.To)
# df["To"] = df["To"].apply(lambda x: x.replace(year=2020, month=1, day=1))

# end = df['To']
# end

### If you want to go back to the very inefficient way of making dataframes

In [ ]:
# def create_dict(df):
#     dictionary = []
#     for row in df.index:
#         start = str(f"'{df.loc[row].at['From']}'")
#         end = str(f"'{df.loc[row].at['To']}'")
#         day = str(f"'{df.loc[row].at['Day']}'")
#         activity = str(f"'{df.loc[row].at['Activity type']}'")
#         dictionary.append(f'dict(Start={start}, End={end}, Day={day}, Activity={activity}),')
#     return dictionary

In [ ]:
# # new_df = create_dict(df)
# new_df = create_dict(df[:100])
# def new_dictionary(new_df):
#     dictionary = []
#     for row in new_df:
#         dictionary = print(row)
#     return dictionary

In [ ]:
# new_dictionary(new_df)

In [ ]:
# new_df = pd.DataFrame([
    
#     dict(Start='2020-01-01 00:42:17', End='2020-01-01 00:47:11', Day='1', Activity='Internet'),
# dict(Start='2020-01-01 00:37:35', End='2020-01-01 00:38:40', Day='1', Activity='Toilet'),
# dict(Start='2020-01-01 23:30:41', End='2020-01-01 00:37:23', Day='366', Activity='Internet'),
# dict(Start='2020-01-01 21:56:39', End='2020-01-01 23:30:40', Day='366', Activity='Games'),
# dict(Start='2020-01-01 21:13:30', End='2020-01-01 21:29:07', Day='366', Activity='Eating'),
# dict(Start='2020-01-01 20:37:14', End='2020-01-01 21:56:37', Day='366', Activity='Internet'),
# dict(Start='2020-01-01 20:31:53', End='2020-01-01 20:37:13', Day='366', Activity='Cooking'),
# dict(Start='2020-01-01 20:29:22', End='2020-01-01 20:30:52', Day='366', Activity='Toilet'),
# dict(Start='2020-01-01 18:25:39', End='2020-01-01 20:29:21', Day='366', Activity='Internet'),
# dict(Start='2020-01-01 18:08:50', End='2020-01-01 18:18:42', Day='366', Activity='Toilet'),
# dict(Start='2020-01-01 09:22:00', End='2020-01-01 18:08:49', Day='366', Activity='Sleeping'),
# dict(Start='2020-01-01 09:19:47', End='2020-01-01 09:21:08', Day='366', Activity='Toilet'),
# dict(Start='2020-01-01 08:15:40', End='2020-01-01 08:30:17', Day='366', Activity='Eating'),
# dict(Start='2020-01-01 07:55:00', End='2020-01-01 09:19:26', Day='366', Activity='Internet'),
# dict(Start='2020-01-01 07:51:12', End='2020-01-01 07:52:01', Day='366', Activity='Toilet'),
# dict(Start='2020-01-01 07:37:34', End='2020-01-01 07:49:48', Day='366', Activity='Walking'),
# dict(Start='2020-01-01 05:57:21', End='2020-01-01 05:58:57', Day='366', Activity='Toilet'),
# dict(Start='2020-01-01 04:08:25', End='2020-01-01 05:55:12', Day='366', Activity='Programming'),
# dict(Start='2020-01-01 03:58:07', End='2020-01-01 04:04:54', Day='366', Activity='Toilet'),
# dict(Start='2020-01-01 03:03:32', End='2020-01-01 03:57:17', Day='366', Activity='Programming'),
# dict(Start='2020-01-01 01:48:08', End='2020-01-01 01:55:26', Day='366', Activity='Eating'),
# dict(Start='2020-01-01 01:44:06', End='2020-01-01 01:46:27', Day='366', Activity='Toilet'),
# dict(Start='2020-01-01 00:31:25', End='2020-01-01 01:42:12', Day='366', Activity='Programming'),
# dict(Start='2020-01-01 00:05:01', End='2020-01-01 00:31:21', Day='366', Activity='Data Analysis'),
# dict(Start='2020-01-01 19:28:46', End='2020-01-01 07:37:33', Day='365', Activity='Working'),
# dict(Start='2020-01-01 19:18:40', End='2020-01-01 19:28:44', Day='365', Activity='Walking'),
# dict(Start='2020-01-01 18:55:15', End='2020-01-01 19:16:49', Day='365', Activity='Internet'),
# dict(Start='2020-01-01 18:32:33', End='2020-01-01 18:55:13', Day='365', Activity='Grooming'),
# dict(Start='2020-01-01 18:31:40', End='2020-01-01 18:32:32', Day='365', Activity='Toilet'),
# dict(Start='2020-01-01 15:28:19', End='2020-01-01 15:59:14', Day='365', Activity='Eating'),
# dict(Start='2020-01-01 15:05:27', End='2020-01-01 18:31:36', Day='365', Activity='Internet'),
# dict(Start='2020-01-01 15:03:15', End='2020-01-01 15:05:26', Day='365', Activity='Cooking'),
# dict(Start='2020-01-01 14:14:36', End='2020-01-01 14:57:16', Day='365', Activity='Sleeping'),
# dict(Start='2020-01-01 14:11:30', End='2020-01-01 14:12:38', Day='365', Activity='Toilet'),
# dict(Start='2020-01-01 12:50:15', End='2020-01-01 14:11:27', Day='365', Activity='Internet'),
# dict(Start='2020-01-01 12:48:08', End='2020-01-01 12:49:58', Day='365', Activity='Toilet'),
# dict(Start='2020-01-01 12:15:26', End='2020-01-01 12:48:07', Day='365', Activity='Internet'),
# dict(Start='2020-01-01 12:14:12', End='2020-01-01 12:15:07', Day='365', Activity='Toilet'),
# dict(Start='2020-01-01 10:42:43', End='2020-01-01 12:14:11', Day='365', Activity='Internet'),
# dict(Start='2020-01-01 09:35:42', End='2020-01-01 10:41:48', Day='365', Activity='Cleaning'),
# dict(Start='2020-01-01 07:51:49', End='2020-01-01 09:35:41', Day='365', Activity='Dishes'),
# dict(Start='2020-01-01 07:48:10', End='2020-01-01 07:49:22', Day='365', Activity='Toilet'),
# dict(Start='2020-01-01 07:40:12', End='2020-01-01 07:48:07', Day='365', Activity='Internet'),
# dict(Start='2020-01-01 03:19:44', End='2020-01-01 07:40:11', Day='365', Activity='Games'),
# dict(Start='2020-01-01 02:39:41', End='2020-01-01 03:17:30', Day='365', Activity='Eating'),
# dict(Start='2020-01-01 02:39:03', End='2020-01-01 03:19:43', Day='365', Activity='Internet'),
# dict(Start='2020-01-01 02:31:44', End='2020-01-01 02:37:05', Day='365', Activity='Toilet'),
# dict(Start='2020-01-01 16:13:21', End='2020-01-01 02:31:42', Day='364', Activity='Sleeping'),
# dict(Start='2020-01-01 15:47:03', End='2020-01-01 16:06:41', Day='364', Activity='Masturbation'),
# dict(Start='2020-01-01 15:41:51', End='2020-01-01 15:44:47', Day='364', Activity='Toilet'),
# dict(Start='2020-01-01 14:08:48', End='2020-01-01 15:39:25', Day='364', Activity='Internet'),
# dict(Start='2020-01-01 09:08:30', End='2020-01-01 14:08:48', Day='364', Activity='Games'),
# dict(Start='2020-01-01 08:40:07', End='2020-01-01 09:00:44', Day='364', Activity='Eating'),
# dict(Start='2020-01-01 08:40:05', End='2020-01-01 09:08:29', Day='364', Activity='Internet'),
# dict(Start='2020-01-01 08:39:11', End='2020-01-01 08:40:01', Day='364', Activity='Toilet'),
# dict(Start='2020-01-01 08:02:31', End='2020-01-01 08:39:09', Day='364', Activity='Internet'),
# dict(Start='2020-01-01 07:55:18', End='2020-01-01 08:02:30', Day='364', Activity='Cooking'),
# dict(Start='2020-01-01 07:44:05', End='2020-01-01 07:53:56', Day='364', Activity='Walking'),
# dict(Start='2020-01-01 06:02:59', End='2020-01-01 06:04:30', Day='364', Activity='Toilet'),
# dict(Start='2020-01-01 03:54:35', End='2020-01-01 03:56:24', Day='364', Activity='Toilet'),
# dict(Start='2020-01-01 02:36:22', End='2020-01-01 03:42:10', Day='364', Activity='Reading'),
# dict(Start='2020-01-01 02:09:15', End='2020-01-01 02:16:03', Day='364', Activity='Eating'),
# dict(Start='2020-01-01 02:02:30', End='2020-01-01 02:04:33', Day='364', Activity='Toilet'),
# dict(Start='2020-01-01 21:18:53', End='2020-01-01 01:56:14', Day='363', Activity='Programming'),
# dict(Start='2020-01-01 19:26:01', End='2020-01-01 19:30:34', Day='363', Activity='Eating'),
# dict(Start='2020-01-01 19:21:52', End='2020-01-01 07:44:04', Day='363', Activity='Working'),
# dict(Start='2020-01-01 19:12:02', End='2020-01-01 19:21:51', Day='363', Activity='Walking'),
# dict(Start='2020-01-01 18:36:19', End='2020-01-01 19:10:35', Day='363', Activity='Grooming'),
# dict(Start='2020-01-01 18:31:54', End='2020-01-01 18:36:18', Day='363', Activity='Toilet'),
# dict(Start='2020-01-01 10:45:20', End='2020-01-01 18:30:31', Day='363', Activity='Sleeping'),
# dict(Start='2020-01-01 10:41:26', End='2020-01-01 10:44:26', Day='363', Activity='Grooming'),
# dict(Start='2020-01-01 10:37:10', End='2020-01-01 10:41:26', Day='363', Activity='Toilet'),
# dict(Start='2020-01-01 10:29:42', End='2020-01-01 10:36:57', Day='363', Activity='Masturbation'),
# dict(Start='2020-01-01 08:09:47', End='2020-01-01 08:23:28', Day='363', Activity='Eating'),
# dict(Start='2020-01-01 08:09:45', End='2020-01-01 10:29:12', Day='363', Activity='Internet'),
# dict(Start='2020-01-01 08:04:29', End='2020-01-01 08:09:44', Day='363', Activity='Cooking'),
# dict(Start='2020-01-01 07:53:14', End='2020-01-01 08:01:35', Day='363', Activity='Walking'),
# dict(Start='2020-01-01 06:02:56', End='2020-01-01 06:22:00', Day='363', Activity='Reading'),
# dict(Start='2020-01-01 06:00:05', End='2020-01-01 06:01:41', Day='363', Activity='Toilet'),
# dict(Start='2020-01-01 04:23:39', End='2020-01-01 05:54:58', Day='363', Activity='Programming'),
# dict(Start='2020-01-01 03:05:25', End='2020-01-01 03:12:13', Day='363', Activity='Eating'),
# dict(Start='2020-01-01 02:09:55', End='2020-01-01 02:38:39', Day='363', Activity='Reading'),
# dict(Start='2020-01-01 01:56:07', End='2020-01-01 01:57:42', Day='363', Activity='Toilet'),
# dict(Start='2020-01-01 00:34:34', End='2020-01-01 01:21:53', Day='363', Activity='Reading'),
# dict(Start='2020-01-01 23:20:31', End='2020-01-01 07:53:14', Day='362', Activity='Working'),
# dict(Start='2020-01-01 23:11:20', End='2020-01-01 23:20:30', Day='362', Activity='Walking'),
# dict(Start='2020-01-01 23:00:30', End='2020-01-01 23:11:18', Day='362', Activity='Grooming'),
# dict(Start='2020-01-01 22:59:38', End='2020-01-01 23:00:29', Day='362', Activity='Toilet'),
# dict(Start='2020-01-01 22:51:51', End='2020-01-01 22:57:47', Day='362', Activity='Eating'),
# dict(Start='2020-01-01 22:51:50', End='2020-01-01 22:59:34', Day='362', Activity='Internet'),
# dict(Start='2020-01-01 22:27:22', End='2020-01-01 22:51:49', Day='362', Activity='Grooming'),
# dict(Start='2020-01-01 22:26:31', End='2020-01-01 22:27:22', Day='362', Activity='Toilet'),
# dict(Start='2020-01-01 20:24:27', End='2020-01-01 22:26:29', Day='362', Activity='Internet'),
# dict(Start='2020-01-01 20:14:51', End='2020-01-01 20:22:34', Day='362', Activity='Toilet'),
# dict(Start='2020-01-01 11:28:59', End='2020-01-01 20:14:43', Day='362', Activity='Sleeping'),
# dict(Start='2020-01-01 11:25:26', End='2020-01-01 11:27:19', Day='362', Activity='Grooming'),
# dict(Start='2020-01-01 11:23:06', End='2020-01-01 11:25:25', Day='362', Activity='Toilet'),
# dict(Start='2020-01-01 07:52:48', End='2020-01-01 08:05:16', Day='362', Activity='Eating'),
# dict(Start='2020-01-01 07:52:24', End='2020-01-01 11:22:59', Day='362', Activity='Internet'),
# dict(Start='2020-01-01 07:45:03', End='2020-01-01 07:52:23', Day='362', Activity='Cooking'),

# ])